<a href="https://colab.research.google.com/github/wanadzhar913/CADS-2022-Bootcamp-JDS-Cohort-2-Capstone-Project/blob/main/data_cleaning_notebooks/Data_Cleaning_May_PKR_Survey_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Note:** This notebook tracks the progress of IVR and Live calls for the May version of the 2023 PKR Tracker Survey.

A few things that differentiate this month's IVR section from the April version: 


In [169]:
import warnings
warnings.filterwarnings("ignore")

# IVR Results

## Import IVR results from folders in Google Drive

In [170]:
import pandas as pd
import numpy as np

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [171]:
# List of state clusters folder
folder_name = [
    'SET 1',
    'SET 2',
    'SET 3',
    'SET 4',
    'SET 5',
    'SET 6',
    'SET 7'
    ]

In [172]:
# Get and store all .csv files path in `csv_path`
import glob
import re

path = "/content/drive/MyDrive/INVOKE Analytics Projects/PKR Survey 2023/Tracking Survey May 2023/IVR Raw Results/"

df_list = []
phonenum_list = []
i = 0
for folder in folder_name:
  
  for fname in glob.glob(path + folder + '/*.csv'):
    
    print(f"Reading in {re.search('Broadcast.*.csv', fname).group()} ..")
    i += 1

    with open(fname, 'r') as f:
      
      df = pd.read_csv(f, skiprows=1, names=range(20), engine='python')

      # Drop all-empty columns
      df.dropna(axis='columns', how='all', inplace=True)

      # Assign first row as column names
      df.columns = df.iloc[0]

      # Select PhoneNo column and all columns from UserKeyPress onwards
      df_phonenum = df[['PhoneNo']]

      df_response = df.loc[:, 'UserKeyPress':]

      df_results = pd.concat([df_phonenum, df_response], axis='columns')

      # Drop rows with blank response in the first question only
      phonenum_recycle = df_results.dropna(subset=['UserKeyPress'])

      # Append the participated phone no. into phonenum_list (select PhoneNo column only)
      phonenum_list.append(phonenum_recycle[['PhoneNo']])

      # Drop incomplete rows
      df_complete = df_results.dropna(axis='index')

      # Reset column names for accurate concatenation later
      df_complete.columns = np.arange(len(df_complete.columns))

      # Initialize `Cluster` column
      df_complete['Region'] = re.search('(\s*K[1-5])', fname).group()

      # Initialize `Set` column
      df_complete['Set'] = folder

      # Select all columns from the first one up to `Cluster`
      df_complete = df_complete.loc[:, :'Set']

      # Filter out key presses that are blank
      df_complete = df_complete.loc[(df_complete.iloc[:, 2].str.len() == 10)]

      # Append the CRs into df_list (to be used later)
      df_list.append(df_complete)

# Combined all participated phone no. stored in phonenum_list
phonenum_combined = pd.concat(phonenum_list, axis='rows')

# Rename column to match with codes in databricks
phonenum_combined.rename(columns={'PhoneNo': 'phonenum'}, inplace=True)

# Inspect result
print('\n')
print(f"Total count of phone no. that need to be excluded in the next sampling: {phonenum_combined.shape[0]}")
print(f"Total files loaded in: {i}")

Reading in Broadcast_List_Report_for_MAY S1 K2 EVENING.csv ..
Reading in Broadcast_List_Report_for_MAY S1 K4 EVENING.csv ..
Reading in Broadcast_List_Report_for_MAY S1 K2 NIGHT.csv ..
Reading in Broadcast_List_Report_for_MAY S1 K4 NIGHT.csv ..
Reading in Broadcast_List_Report_for_MAY S1 K2 NOON .csv ..
Reading in Broadcast_List_Report_for_MAY S1 K3 EVENING.csv ..
Reading in Broadcast_List_Report_for_MAY S1 K3 NOON .csv ..
Reading in Broadcast_List_Report_for_MAY S1 K3 NIGHT.csv ..
Reading in Broadcast_List_Report_for_MAY S1 K4 NOON .csv ..
Reading in Broadcast_List_Report_for_MAY S1 K5 EVENING.csv ..
Reading in Broadcast_List_Report_for_MAY S1 K5 NIGHT.csv ..
Reading in Broadcast_List_Report_for_MAY S1 K5 NOON.csv ..
Reading in Broadcast_List_Report_for_MAY S1 K1 EVENING.csv ..
Reading in Broadcast_List_Report_for_MAY S1 K1 NOON.csv ..
Reading in Broadcast_List_Report_for_MAY S1 K1 NIGHT.csv ..
Reading in Broadcast_List_Report_for_MAY S2 K1 NOON .csv ..
Reading in Broadcast_List_Report

In [173]:
from datetime import date

"""
since CC labels the folders by date, we'll need to specify a 
variable with today's date to get the latest file. The last day
where we read the files in was on the 28-3-2023
"""
today = date.today()
formatted_date = "/" + today.strftime("%d-%m-%Y").replace("-0", "-")
print("Today's date:", formatted_date)

Today's date: /01-6-2023


In [174]:
# Export phone no. list as csv to be uploaded into Databricks, then to be excluded in the next sampling
phonenum_combined.to_csv('ivr_may_pkr_2023_used_phonenum_v{}.csv'.format(formatted_date.replace('/', '')), index=False)

## Merge all raw IVR results

In [175]:
df_merge = pd.concat(df_list, axis='index')

print(df_merge.shape)
print('\n')
print(f'Sebastian IVR count by Region as of {today.strftime("%d-%m-%Y").replace("-0", "-")}')
print(df_merge['Region'].value_counts())
print('\n')
print(f'Sebastian IVR count by Set as of {today.strftime("%d-%m-%Y").replace("-0", "-")}')
print(df_merge['Set'].value_counts())

(3385, 15)


Sebastian IVR count by Region as of 01-6-2023
 K3    939
 K1    859
 K2    777
 K4    603
 K5    207
Name: Region, dtype: int64


Sebastian IVR count by Set as of 01-6-2023
SET 1    735
SET 3    698
SET 4    608
SET 6    423
SET 2    385
SET 7    356
SET 5    180
Name: Set, dtype: int64


## Data Cleaning & Preprocessing

### Separate CRs by Set

In [176]:
set1 = df_merge.loc[df_merge['Set'] == 'SET 1']

set2 = df_merge.loc[df_merge['Set'] == 'SET 2']

set3 = df_merge.loc[df_merge['Set'] == 'SET 3']

set4 = df_merge.loc[df_merge['Set'] == 'SET 4']

set5 = df_merge.loc[df_merge['Set'] == 'SET 5']

set6 = df_merge.loc[df_merge['Set'] == 'SET 6']

set7 = df_merge.loc[df_merge['Set'] == 'SET 7']

### Create Region-State mapping

In [177]:
# Region-state mapping

# northeast
kluster1 = {'FlowNo_2=1': 'Kelantan',
            'FlowNo_2=2': 'Terengganu',
            'FlowNo_2=3': 'Pahang',
            'FlowNo_2=4': 'Lain-lain'}

# northwest
kluster2 = {'FlowNo_2=1': 'Kedah',
            'FlowNo_2=2': 'Pulau Pinang',
            'FlowNo_2=3': 'Perlis',
            'FlowNo_2=4': 'Lain-lain'}

# central
kluster3 = {'FlowNo_2=1': 'Perak',
            'FlowNo_2=2': 'Selangor',
            'FlowNo_2=3': 'Kuala Lumpur',
            'FlowNo_2=4': 'Negeri Sembilan',
            'FlowNo_2=5': 'Lain-lain'}

# south
kluster4 = {'FlowNo_2=1': 'Melaka',
            'FlowNo_2=2': 'Johor',
            'FlowNo_2=4': 'Lain-lain'}

# east
kluster5 = {'FlowNo_2=1': 'Sabah',
            'FlowNo_2=2': 'Sarawak',
            'FlowNo_2=4': 'Lain-lain'}

### Create Region-Ethnic mapping

In [178]:
# Region-ethnic mapping
peninsular_eth = {'1': 'MELAYU',
                  '2': 'CINA',
                  '3': 'INDIA',
                  '4': 'LAIN-LAIN',
                  '5': 'MELAYU'}  # Handling for potentially incorrect key press (mode imputation)

east_eth = {'1': 'MELAYU',
            '2': 'CINA',
            '3': 'MUSLIM BUMIPUTERA',
            '4': 'NON-MUSLIM BUMIPUTERA',
            '5': 'LAIN-LAIN'}

### SET 1

In [179]:
set1.head(3)

,0,1,2,3,4,5,Region,Set,6,7,8,9,10,11,12
100,60134892545,FlowNo_2=2,FlowNo_3=3,FlowNo_4=1,FlowNo_5=1,FlowNo_6=2,K2,SET 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
806,60124710546,FlowNo_2=2,FlowNo_3=4,FlowNo_4=3,FlowNo_5=2,FlowNo_6=2,K2,SET 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1075,60104604287,FlowNo_2=1,FlowNo_3=1,FlowNo_4=1,FlowNo_5=2,FlowNo_6=2,K2,SET 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
# Map responses accordingly (But, for state & ethnic questions have to be mapped conditionally, based on the Region):

# 1. Which of the following issues is most important to you?
q1_mapping = {'FlowNo_3=1': 'Cost of living and job opportunities',
              'FlowNo_3=2': 'Corruption',
              'FlowNo_3=3': 'Political stability',
              'FlowNo_3=4': 'Strength of national economy',
              'FlowNo_3=5': 'Quality of national education'}

# 2. Are you satisfied with the current economic situation?
q2_mapping = {'FlowNo_4=1': 'Yes',
              'FlowNo_4=2': 'No',
              'FlowNo_4=3': 'Unsure'}

gender = {'FlowNo_6=1': 'L ',
          'FlowNo_6=2': 'P '}

# Total CR
print(f"Total CR for Set 1: {set1.shape[0]}")

# Select relevant columns only
set1 = set1.loc[:, :'Set']
  
# Rename columns
set1.columns = ['phonenum',
                'State',
                '1. Which of the following issues is most important to you?',
                '2. Are you satisfied with the current economic situation?',
                'ethgroup',
                'gender',
                'Region',
                'Set']

# Map response value
set1['1. Which of the following issues is most important to you?'].replace(q1_mapping, inplace=True)

set1['2. Are you satisfied with the current economic situation?'].replace(q2_mapping, inplace=True)

set1['gender'].replace(gender, inplace=True)

set1.head(3)

Total CR for Set 1: 735


,phonenum,State,1. Which of the following issues is most important to you?,2. Are you satisfied with the current economic situation?,ethgroup,gender,Region,Set
100,60134892545,FlowNo_2=2,Political stability,Yes,FlowNo_5=1,P,K2,SET 1
806,60124710546,FlowNo_2=2,Strength of national economy,Unsure,FlowNo_5=2,P,K2,SET 1
1075,60104604287,FlowNo_2=1,Cost of living and job opportunities,Yes,FlowNo_5=2,P,K2,SET 1


In [181]:
# Assign State values according to Region
set1.loc[set1['Region'] == ' K1', 'State'] = set1.loc[set1['Region'] == ' K1', 'State'].replace(kluster1)

set1.loc[set1['Region'] == ' K2', 'State'] = set1.loc[set1['Region'] == ' K2', 'State'].replace(kluster2)

set1.loc[set1['Region'] == ' K3', 'State'] = set1.loc[set1['Region'] == ' K3', 'State'].replace(kluster3)

set1.loc[set1['Region'] == ' K4', 'State'] = set1.loc[set1['Region'] == ' K4', 'State'].replace(kluster4)

set1.loc[set1['Region'] == ' K5', 'State'] = set1.loc[set1['Region'] == ' K5', 'State'].replace(kluster5)

In [182]:
# Assign Ethnic values according to Region

# Extract only the key press value in ethgroup column
set1['ethgroup_key'] = set1['ethgroup'].str[-1]

set1.loc[set1['Region'] != ' K5', 'ethgroup'] = set1.loc[set1['Region'] != ' K5', 'ethgroup_key'].replace(peninsular_eth)

set1.loc[set1['Region'] == ' K5', 'ethgroup'] = set1.loc[set1['Region'] == ' K5', 'ethgroup_key'].replace(east_eth)

set1.drop(['ethgroup_key'], axis='columns', inplace=True)

In [183]:
# drop rows where all columns still contain the string pattern 'FlowNo_'
set1 = set1[~set1.apply(lambda x: x.str.contains('FlowNo_')).any(axis=1)]

In [184]:
# drop null values in gender. Weird, it's only for `set1`.
set1 = set1.dropna(subset=['gender'])

In [185]:
set1.head(3)

,phonenum,State,1. Which of the following issues is most important to you?,2. Are you satisfied with the current economic situation?,ethgroup,gender,Region,Set
100,60134892545,Pulau Pinang,Political stability,Yes,MELAYU,P,K2,SET 1
806,60124710546,Pulau Pinang,Strength of national economy,Unsure,CINA,P,K2,SET 1
1075,60104604287,Kedah,Cost of living and job opportunities,Yes,CINA,P,K2,SET 1


In [186]:
# final check
# for x in set1.columns:
#   print(set1[x].value_counts(normalize=True))
#   print("\n")

In [187]:
# Export as .xlsx file
# set1.to_excel('ivr_march_pkr2023_set1_cleaned.xlsx', index=False)

### SET 2

In [188]:
set2.head(3)

,0,1,2,3,4,5,Region,Set,6,7,8,9,10,11,12
351,60129885458,FlowNo_2=1,FlowNo_3=1,FlowNo_4=1,FlowNo_5=1,FlowNo_6=1,K1,SET 2,FlowNo_7=1,FlowNo_8=1,FlowNo_9=1,FlowNo_10=1,FlowNo_11=1,FlowNo_12=1,FlowNo_13=1
1285,60145124215,FlowNo_2=2,FlowNo_3=1,FlowNo_4=5,FlowNo_5=1,FlowNo_6=5,K1,SET 2,FlowNo_7=1,FlowNo_8=1,FlowNo_9=1,FlowNo_10=1,FlowNo_11=1,FlowNo_12=1,FlowNo_13=1
1869,60179846717,FlowNo_2=1,FlowNo_3=5,FlowNo_4=1,FlowNo_5=5,FlowNo_6=1,K1,SET 2,FlowNo_7=5,FlowNo_8=1,FlowNo_9=2,FlowNo_10=1,FlowNo_11=5,FlowNo_12=1,FlowNo_13=1


In [189]:
# Map responses accordingly (But, for state & ethnic questions have to be mapped conditionally, based on the Region):

# 4a. Anwar Ibrahim
q4a_mapping = {'FlowNo_3=1': '1) Very negative',
              'FlowNo_3=2': '2) Negative',
              'FlowNo_3=3': '3) Neutral',
              'FlowNo_3=4': '4) Positive',
              'FlowNo_3=5': '5) Very positive'}

# 4b. Muhyiddin Yassin
q4b_mapping = {'FlowNo_4=1': '1) Very negative',
              'FlowNo_4=2': '2) Negative',
              'FlowNo_4=3': '3) Neutral',
              'FlowNo_4=4': '4) Positive',
              'FlowNo_4=5': '5) Very positive'}

# 4c. Zahid Hamidi
q4c_mapping = {'FlowNo_5=1': '1) Very negative',
              'FlowNo_5=2': '2) Negative',
              'FlowNo_5=3': '3) Neutral',
              'FlowNo_5=4': '4) Positive',
              'FlowNo_5=5': '5) Very positive'}

# 4d. Hadi Awang
q4d_mapping = {'FlowNo_6=1': '1) Very negative',
              'FlowNo_6=2': '2) Negative',
              'FlowNo_6=3': '3) Neutral',
              'FlowNo_6=4': '4) Positive',
              'FlowNo_6=5': '5) Very positive'}

In [190]:
# 4e. Fadhilah Yusof
q4e_mapping = {'FlowNo_7=1': '1) Very negative',
              'FlowNo_7=2': '2) Negative',
              'FlowNo_7=3': '3) Neutral',
              'FlowNo_7=4': '4) Positive',
              'FlowNo_7=5': '5) Very positive'}

# 4f. Rafizi Ramli
q4f_mapping = {'FlowNo_8=1': '1) Very negative',
              'FlowNo_8=2': '2) Negative',
              'FlowNo_8=3': '3) Neutral',
              'FlowNo_8=4': '4) Positive',
              'FlowNo_8=5': '5) Very positive'}

# 4g. Anthony Loke
q4g_mapping = {'FlowNo_9=1': '1) Very negative',
              'FlowNo_9=2': '2) Negative',
              'FlowNo_9=3': '3) Neutral',
              'FlowNo_9=4': '4) Positive',
              'FlowNo_9=5': '5) Very positive'}

# 4h. Mohamad Hasan
q4h_mapping = {'FlowNo_10=1': '1) Very negative',
              'FlowNo_10=2': '2) Negative',
              'FlowNo_10=3': '3) Neutral',
              'FlowNo_10=4': '4) Positive',
              'FlowNo_10=5': '5) Very positive'}

# 4i. Mohamad Sabu
q4i_mapping = {'FlowNo_11=1': '1) Very negative',
              'FlowNo_11=2': '2) Negative',
              'FlowNo_11=3': '3) Neutral',
              'FlowNo_11=4': '4) Positive',
              'FlowNo_11=5': '5) Very positive'}

In [191]:
gender = {'FlowNo_13=1': 'L ',
          'FlowNo_13=2': 'P '}

# Total CR
print(f"Total CR for Set 2: {set2.shape[0]}")

# Select relevant columns only
set2.dropna(axis='columns', inplace=True)
  
# Rename columns
set2.columns = ['phonenum',
                'State',
                '4a. Anwar Ibrahim',
                '4b. Muhyiddin Yassin',
                '4c. Zahid Hamidi',
                '4d. Hadi Awang',
                'Region',
                'Set',
                '4e. Fadhilah Yusof',
                '4f. Rafizi Ramli',
                '4g. Anthony Loke',
                '4h. Mohamad Hasan',
                '4i. Mohamad Sabu',
                'ethgroup',
                'gender']

# Map response value
set2['4a. Anwar Ibrahim'].replace(q4a_mapping, inplace=True)
set2['4b. Muhyiddin Yassin'].replace(q4b_mapping, inplace=True)
set2['4c. Zahid Hamidi'].replace(q4c_mapping, inplace=True)
set2['4d. Hadi Awang'].replace(q4d_mapping, inplace=True)
set2['4e. Fadhilah Yusof'].replace(q4e_mapping, inplace=True)
set2['4f. Rafizi Ramli'].replace(q4f_mapping, inplace=True)
set2['4g. Anthony Loke'].replace(q4g_mapping, inplace=True)
set2['4h. Mohamad Hasan'].replace(q4h_mapping, inplace=True)
set2['4i. Mohamad Sabu'].replace(q4i_mapping, inplace=True)

# reset column order
set2 = set2[['phonenum',
              'State',
              '4a. Anwar Ibrahim',
              '4b. Muhyiddin Yassin',
              '4c. Zahid Hamidi',
              '4d. Hadi Awang',
              '4e. Fadhilah Yusof',
              '4f. Rafizi Ramli',
              '4g. Anthony Loke',
              '4h. Mohamad Hasan',
              '4i. Mohamad Sabu',
              'ethgroup',
              'Region',
              'Set',
              'gender']]

set2['gender'].replace(gender, inplace=True)

set2.head(3)

Total CR for Set 2: 385


,phonenum,State,4a. Anwar Ibrahim,4b. Muhyiddin Yassin,4c. Zahid Hamidi,4d. Hadi Awang,4e. Fadhilah Yusof,4f. Rafizi Ramli,4g. Anthony Loke,4h. Mohamad Hasan,4i. Mohamad Sabu,ethgroup,Region,Set,gender
351,60129885458,FlowNo_2=1,1) Very negative,1) Very negative,1) Very negative,1) Very negative,1) Very negative,1) Very negative,1) Very negative,1) Very negative,1) Very negative,FlowNo_12=1,K1,SET 2,L
1285,60145124215,FlowNo_2=2,1) Very negative,5) Very positive,1) Very negative,5) Very positive,1) Very negative,1) Very negative,1) Very negative,1) Very negative,1) Very negative,FlowNo_12=1,K1,SET 2,L
1869,60179846717,FlowNo_2=1,5) Very positive,1) Very negative,5) Very positive,1) Very negative,5) Very positive,1) Very negative,2) Negative,1) Very negative,5) Very positive,FlowNo_12=1,K1,SET 2,L


In [192]:
# Assign State values according to Region
set2.loc[set2['Region'] == ' K1', 'State'] = set2.loc[set2['Region'] == ' K1', 'State'].replace(kluster1)

set2.loc[set2['Region'] == ' K2', 'State'] = set2.loc[set2['Region'] == ' K2', 'State'].replace(kluster2)

set2.loc[set2['Region'] == ' K3', 'State'] = set2.loc[set2['Region'] == ' K3', 'State'].replace(kluster3)

set2.loc[set2['Region'] == ' K4', 'State'] = set2.loc[set2['Region'] == ' K4', 'State'].replace(kluster4)

set2.loc[set2['Region'] == ' K5', 'State'] = set2.loc[set2['Region'] == ' K5', 'State'].replace(kluster5)

In [193]:
# Assign Ethnic values according to Region

# Extract only the key press value in ethgroup column
set2['ethgroup_key'] = set2['ethgroup'].str[-1]

set2.loc[set2['Region'] != ' K5', 'ethgroup'] = set2.loc[set2['Region'] != ' K5', 'ethgroup_key'].replace(peninsular_eth)

set2.loc[set2['Region'] == ' K5', 'ethgroup'] = set2.loc[set2['Region'] == ' K5', 'ethgroup_key'].replace(east_eth)

set2.drop(['ethgroup_key'], axis='columns', inplace=True)

set2.head(3)

,phonenum,State,4a. Anwar Ibrahim,4b. Muhyiddin Yassin,4c. Zahid Hamidi,4d. Hadi Awang,4e. Fadhilah Yusof,4f. Rafizi Ramli,4g. Anthony Loke,4h. Mohamad Hasan,4i. Mohamad Sabu,ethgroup,Region,Set,gender
351,60129885458,Kelantan,1) Very negative,1) Very negative,1) Very negative,1) Very negative,1) Very negative,1) Very negative,1) Very negative,1) Very negative,1) Very negative,MELAYU,K1,SET 2,L
1285,60145124215,Terengganu,1) Very negative,5) Very positive,1) Very negative,5) Very positive,1) Very negative,1) Very negative,1) Very negative,1) Very negative,1) Very negative,MELAYU,K1,SET 2,L
1869,60179846717,Kelantan,5) Very positive,1) Very negative,5) Very positive,1) Very negative,5) Very positive,1) Very negative,2) Negative,1) Very negative,5) Very positive,MELAYU,K1,SET 2,L


In [194]:
# drop rows where all columns still contain the string pattern 'FlowNo_'
set2 = set2[~set2.apply(lambda x: x.str.contains('FlowNo_')).any(axis=1)]

In [195]:
# final check
# for x in set2.columns:
#   print(set2[x].value_counts(normalize=True))
#   print("\n")

In [196]:
# Export as .xlsx file
# set2.to_excel('ivr_march_pkr2023_set2_cleaned.xlsx', index=False)

### SET 3

In [197]:
print(f'Shape: {set3.shape}')
set3.head(3)

Shape: (698, 15)


,0,1,2,3,4,5,Region,Set,6,7,8,9,10,11,12
80,60134245206,FlowNo_2=2,FlowNo_3=3,FlowNo_4=,FlowNo_5=,FlowNo_6=1,K1,SET 3,FlowNo_7=1,NaN,NaN,NaN,NaN,NaN,NaN
297,60199566453,FlowNo_2=2,FlowNo_3=1,FlowNo_4=4,FlowNo_5=,FlowNo_6=1,K1,SET 3,FlowNo_7=1,NaN,NaN,NaN,NaN,NaN,NaN
940,60133493499,FlowNo_2=3,FlowNo_3=1,FlowNo_4=3,FlowNo_5=,FlowNo_6=1,K1,SET 3,FlowNo_7=1,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
# Map responses accordingly (But, for state & ethnic questions have to be mapped conditionally, based on the Region):

# 5. Are you satisfied with the performance of the current government?
q5_mapping = {'FlowNo_3=1': 'Yes',
              'FlowNo_3=2': 'No',
              'FlowNo_3=3': 'Unsure'}

# 6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?
q6_mapping = {'FlowNo_4=1': 'Improvement of racial and religious situation',
               'FlowNo_4=2': 'Improvement of federal infrastructure',
               'FlowNo_4=3': 'Successfully reducing corruption',
               'FlowNo_4=4': 'Increased economic opportunities',
               'FlowNo_4=5': 'Performing leaders or representatives',
               'FlowNo_4=': ''}

# 7.[IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?
q7_mapping = {'FlowNo_5=1': 'Worsening of racial and religious situation',
               'FlowNo_5=2': 'Neglect of federal infrastructure',
               'FlowNo_5=3': 'Failure to reduce corruption',
               'FlowNo_5=4': 'Failure to create economic opportunities',
               'FlowNo_5=5': 'Poor leaders or representatives',
               'FlowNo_5=': ''}

gender = {'FlowNo_7=1': 'L ',
          'FlowNo_7=2': 'P '}

# Total CR
print(f"Total CR for Set 3: {set3.shape[0]}")

# Select relevant columns only
set3.dropna(axis='columns', inplace=True)
  
# Rename columns
set3.columns = ['phonenum',
                'State',
                '5. Are you satisfied with the performance of the current government?',
                "6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?",
                "7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?",
                'ethgroup',
                'Region',
                'Set',
                'gender']

# Map response value
set3['5. Are you satisfied with the performance of the current government?'].replace(q5_mapping, inplace=True)

set3['6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?'].replace(q6_mapping, inplace=True)

set3['7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?'].replace(q7_mapping, inplace=True)

set3['gender'].replace(gender, inplace=True)

set3.head(3)

Total CR for Set 3: 698


,phonenum,State,5. Are you satisfied with the performance of the current government?,6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?,7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?,ethgroup,Region,Set,gender
80,60134245206,FlowNo_2=2,Unsure,,,FlowNo_6=1,K1,SET 3,L
297,60199566453,FlowNo_2=2,Yes,Increased economic opportunities,,FlowNo_6=1,K1,SET 3,L
940,60133493499,FlowNo_2=3,Yes,Successfully reducing corruption,,FlowNo_6=1,K1,SET 3,L


In [199]:
# Assign State values according to Region
set3.loc[set3['Region'] == ' K1', 'State'] = set3.loc[set3['Region'] == ' K1', 'State'].replace(kluster1)

set3.loc[set3['Region'] == ' K2', 'State'] = set3.loc[set3['Region'] == ' K2', 'State'].replace(kluster2)

set3.loc[set3['Region'] == ' K3', 'State'] = set3.loc[set3['Region'] == ' K3', 'State'].replace(kluster3)

set3.loc[set3['Region'] == ' K4', 'State'] = set3.loc[set3['Region'] == ' K4', 'State'].replace(kluster4)

set3.loc[set3['Region'] == ' K5', 'State'] = set3.loc[set3['Region'] == ' K5', 'State'].replace(kluster5)

In [200]:
# Assign Ethnic values according to Region

# Extract only the key press value in ethgroup column
set3['ethgroup_key'] = set3['ethgroup'].str[-1]

set3.loc[set3['Region'] != ' K5', 'ethgroup'] = set3.loc[set3['Region'] != ' K5', 'ethgroup_key'].replace(peninsular_eth)

set3.loc[set3['Region'] == ' K5', 'ethgroup'] = set3.loc[set3['Region'] == ' K5', 'ethgroup_key'].replace(east_eth)

set3.drop(['ethgroup_key'], axis='columns', inplace=True)

In [201]:
# Manual inspection shows several entries violate skip logic. We'll manually remove this. 

# If q5. == 'Yes', then q7. ought to be empty
set3.loc[set3['5. Are you satisfied with the performance of the current government?'] == 'Yes',
              "7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?"] = set3.loc[set3['5. Are you satisfied with the performance of the current government?'] == 'Yes',
              "7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?"].replace("", inplace=True)

# If q5. == 'No', then q6. ought to be empty
set3.loc[set3['5. Are you satisfied with the performance of the current government?'] == 'No',
              "6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?"] = set3.loc[set3['5. Are you satisfied with the performance of the current government?'] == 'No',
              "6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?"].replace("", inplace=True)

# If q5. == 'Unsure', then q6. & q7. ought to be empty
set3.loc[set3['5. Are you satisfied with the performance of the current government?'] == 'Unsure',
              ["6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?",
               "7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?"]] = set3.loc[set3['5. Are you satisfied with the performance of the current government?'] == 'Unsure',
              ["6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?",
               "7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?"]].replace("", inplace=True)

In [202]:
# drop rows where columns still contain the string pattern 'FlowNo_'
set3 = set3[~set3.apply(lambda x: x.str.contains('FlowNo_')).any(axis=1)]

In [203]:
# final check
# for x in set3.columns:
#   print(set3[x].value_counts(normalize=True))
#   print("\n")

In [204]:
print(f'Shape: {set3.shape}')
set3.head(3)

Shape: (682, 9)


,phonenum,State,5. Are you satisfied with the performance of the current government?,6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?,7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?,ethgroup,Region,Set,gender
80,60134245206,Terengganu,Unsure,None,None,MELAYU,K1,SET 3,L
297,60199566453,Terengganu,Yes,Increased economic opportunities,None,MELAYU,K1,SET 3,L
940,60133493499,Pahang,Yes,Successfully reducing corruption,None,MELAYU,K1,SET 3,L


In [205]:
# Export as .xlsx file
# set3.to_excel('ivr_pkr2023_set3_cleaned.xlsx', index=False)

### SET 4

In [206]:
set4.head(3)

,0,1,2,3,4,5,Region,Set,6,7,8,9,10,11,12
74,601136277810,FlowNo_2=1,FlowNo_3=2,FlowNo_4=2,FlowNo_5=1,FlowNo_6=1,K2,SET 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,60194337371,FlowNo_2=1,FlowNo_3=1,FlowNo_4=3,FlowNo_5=1,FlowNo_6=1,K2,SET 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,60164827158,FlowNo_2=2,FlowNo_3=2,FlowNo_4=2,FlowNo_5=1,FlowNo_6=1,K2,SET 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
# Map responses accordingly (But, for state & ethnic questions have to be mapped conditionally, based on the Region):

# 8. Are you satisfied with the performance of the Prime Minister?
q8_mapping = {'FlowNo_3=1': 'Yes',
              'FlowNo_3=2': 'No',
              'FlowNo_3=3': 'Unsure'}

# 9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?
q9_mapping = {'FlowNo_4=1': 'Unity Gov.',
              'FlowNo_4=2': 'PN-BN',
              'FlowNo_4=3': 'About the same'}

gender = {'FlowNo_6=1': 'L ',
          'FlowNo_6=2': 'P '}

# Total CR
print(f"Total CR for Set 4: {set4.shape[0]}")

# Select relevant columns only
set4.dropna(axis='columns', inplace=True)
  
# Rename columns
set4.columns = ['phonenum',
                'State',
                '8. Are you satisfied with the performance of the Prime Minister?',
                '9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?',
                'ethgroup',
                'gender',
                'Region',
                'Set']

# Map response value
set4['8. Are you satisfied with the performance of the Prime Minister?'].replace(q8_mapping, inplace=True)

set4['9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?'].replace(q9_mapping, inplace=True)

set4['gender'].replace(gender, inplace=True)

set4.head(3)

Total CR for Set 4: 608


,phonenum,State,8. Are you satisfied with the performance of the Prime Minister?,"9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?",ethgroup,gender,Region,Set
74,601136277810,FlowNo_2=1,No,PN-BN,FlowNo_5=1,L,K2,SET 4
146,60194337371,FlowNo_2=1,Yes,About the same,FlowNo_5=1,L,K2,SET 4
268,60164827158,FlowNo_2=2,No,PN-BN,FlowNo_5=1,L,K2,SET 4


In [208]:
# Assign State values according to Region
set4.loc[set4['Region'] == ' K1', 'State'] = set4.loc[set4['Region'] == ' K1', 'State'].replace(kluster1)

set4.loc[set4['Region'] == ' K2', 'State'] = set4.loc[set4['Region'] == ' K2', 'State'].replace(kluster2)

set4.loc[set4['Region'] == ' K3', 'State'] = set4.loc[set4['Region'] == ' K3', 'State'].replace(kluster3)

set4.loc[set4['Region'] == ' K4', 'State'] = set4.loc[set4['Region'] == ' K4', 'State'].replace(kluster4)

set4.loc[set4['Region'] == ' K5', 'State'] = set4.loc[set4['Region'] == ' K5', 'State'].replace(kluster5)

In [209]:
# Assign Ethnic values according to Region

# Extract only the key press value in ethgroup column
set4['ethgroup_key'] = set4['ethgroup'].str[-1]

set4.loc[set4['Region'] != ' K5', 'ethgroup'] = set4.loc[set4['Region'] != ' K5', 'ethgroup_key'].replace(peninsular_eth)

set4.loc[set4['Region'] == ' K5', 'ethgroup'] = set4.loc[set4['Region'] == ' K5', 'ethgroup_key'].replace(east_eth)

set4.drop(['ethgroup_key'], axis='columns', inplace=True)

In [210]:
# drop rows where all columns still contain the string pattern 'FlowNo_'
set4 = set4[~set4.apply(lambda x: x.str.contains('FlowNo_')).any(axis=1)]

In [211]:
set4.head(3)

,phonenum,State,8. Are you satisfied with the performance of the Prime Minister?,"9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?",ethgroup,gender,Region,Set
74,601136277810,Kedah,No,PN-BN,MELAYU,L,K2,SET 4
146,60194337371,Kedah,Yes,About the same,MELAYU,L,K2,SET 4
268,60164827158,Pulau Pinang,No,PN-BN,MELAYU,L,K2,SET 4


In [212]:
# final check
# for x in set4.columns:
#   print(set4[x].value_counts(normalize=True))
#   print("\n")

In [213]:
# Export as .xlsx file
# set4.to_excel('ivr_pkr2023_set4_cleaned.xlsx', index=False)

### SET 5

In [214]:
set5.head(3)

,0,1,2,3,4,5,Region,Set,6,7,8,9,10,11,12
1338,60199014607,FlowNo_2=3,FlowNo_3=3,FlowNo_4=,FlowNo_5=,FlowNo_6=1,K1,SET 5,FlowNo_7=2,NaN,NaN,NaN,NaN,NaN,NaN
1788,60123944869,FlowNo_2=3,FlowNo_3=3,FlowNo_4=,FlowNo_5=,FlowNo_6=1,K1,SET 5,FlowNo_7=1,NaN,NaN,NaN,NaN,NaN,NaN
3063,60148225957,FlowNo_2=1,FlowNo_3=3,FlowNo_4=,FlowNo_5=,FlowNo_6=1,K1,SET 5,FlowNo_7=1,NaN,NaN,NaN,NaN,NaN,NaN


In [215]:
# Map responses accordingly (But, for state & ethnic questions have to be mapped conditionally, based on the Region):

# 11. Are you satisfied with the current State Government’s performance?
q11_mapping = {'FlowNo_3=1': 'Yes',
              'FlowNo_3=2': 'No',
              'FlowNo_3=3': 'Unsure'}

# 12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?
q12_mapping = {'FlowNo_4=1': 'Empowerment of Islam and Malay-Muslim institutions',
              'FlowNo_4=2': 'Improvement of local infrastructure',
              'FlowNo_4=3': 'Success at addressing floods and other crises',
              'FlowNo_4=4': 'Increased economic opportunities',
              'FlowNo_4=5': 'Performing leaders or representatives',
              'FlowNo_4=': ""}

# 13.[IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?
q13_mapping = {'FlowNo_5=1': 'Neglect of Islam and Malay-Muslim institution',
              'FlowNo_5=2': 'Neglect of local infrastructure',
              'FlowNo_5=3': 'Failure to prepare for floods and other crises',
              'FlowNo_5=4': 'Failure to create economic opportunities',
              'FlowNo_5=5': 'Poor leaders or representatives',
              'FlowNo_5=': ""}

gender = {'FlowNo_7=1': 'L ',
          'FlowNo_7=2': 'P '}

# Total CR
print(f"Total CR for Set 5: {set5.shape[0]}")

# Select relevant columns only
set5.dropna(axis='columns', inplace=True)
  
# Rename columns
set5.columns = ['phonenum',
                'State',
                "11. Are you satisfied with the current State Government’s performance?",
                "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?",
                "13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?",
                'ethgroup',
                'Region',
                'Set',
                'gender']

# Map response value
set5["11. Are you satisfied with the current State Government’s performance?"].replace(q11_mapping, inplace=True)

set5["12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?"].replace(q12_mapping, inplace=True)

set5["13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?"].replace(q13_mapping, inplace=True)

set5['gender'].replace(gender, inplace=True)

set5.head(3)

Total CR for Set 5: 180


,phonenum,State,11. Are you satisfied with the current State Government’s performance?,12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?,13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?,ethgroup,Region,Set,gender
1338,60199014607,FlowNo_2=3,Unsure,,,FlowNo_6=1,K1,SET 5,P
1788,60123944869,FlowNo_2=3,Unsure,,,FlowNo_6=1,K1,SET 5,L
3063,60148225957,FlowNo_2=1,Unsure,,,FlowNo_6=1,K1,SET 5,L


In [216]:
# Assign State values according to Region
set5.loc[set5['Region'] == ' K1', 'State'] = set5.loc[set5['Region'] == ' K1', 'State'].replace(kluster1)

set5.loc[set5['Region'] == ' K2', 'State'] = set5.loc[set5['Region'] == ' K2', 'State'].replace(kluster2)

set5.loc[set5['Region'] == ' K3', 'State'] = set5.loc[set5['Region'] == ' K3', 'State'].replace(kluster3)

set5.loc[set5['Region'] == ' K4', 'State'] = set5.loc[set5['Region'] == ' K4', 'State'].replace(kluster4)

set5.loc[set5['Region'] == ' K5', 'State'] = set5.loc[set5['Region'] == ' K5', 'State'].replace(kluster5)

In [217]:
# Assign Ethnic values according to Region

# Extract only the key press value in ethgroup column
set5['ethgroup_key'] = set5['ethgroup'].str[-1]

set5.loc[set5['Region'] != ' K5', 'ethgroup'] = set5.loc[set5['Region'] != ' K5', 'ethgroup_key'].replace(peninsular_eth)

set5.loc[set5['Region'] == ' K5', 'ethgroup'] = set5.loc[set5['Region'] == ' K5', 'ethgroup_key'].replace(east_eth)

set5.drop(['ethgroup_key'], axis='columns', inplace=True)

In [218]:
# Just in case any entries violate skip logic. We'll manually remove this. 

# If q11. == 'Yes', then q13. ought to be empty
set5.loc[set5["11. Are you satisfied with the current State Government’s performance?"] == 'Yes',
              "13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?"] = set5.loc[set5["11. Are you satisfied with the current State Government’s performance?"] == 'Yes',
              "13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?"].replace("", inplace=True)

# If q11. == 'No', then q12. ought to be empty
set5.loc[set5["11. Are you satisfied with the current State Government’s performance?"] == 'No',
              "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?"] = set5.loc[set5["11. Are you satisfied with the current State Government’s performance?"] == 'No',
              "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?"].replace("", inplace=True)

# If q11. == 'Unsure', then q12. & q13. ought to be empty
set5.loc[set5["11. Are you satisfied with the current State Government’s performance?"] == 'Unsure',
              ["13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?",
              "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?"]] = set5.loc[set5["11. Are you satisfied with the current State Government’s performance?"] == 'Unsure',
              ["13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?",
              "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?"]].replace("", inplace=True)

In [219]:
# drop rows where all columns still contain the string pattern 'FlowNo_'
set5 = set5[~set5.apply(lambda x: x.str.contains('FlowNo_')).any(axis=1)]

In [220]:
set5.head(3)

,phonenum,State,11. Are you satisfied with the current State Government’s performance?,12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?,13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?,ethgroup,Region,Set,gender
1338,60199014607,Pahang,Unsure,None,None,MELAYU,K1,SET 5,P
1788,60123944869,Pahang,Unsure,None,None,MELAYU,K1,SET 5,L
3063,60148225957,Kelantan,Unsure,None,None,MELAYU,K1,SET 5,L


In [221]:
# final check
for x in set5.columns:
  print(set5[x].value_counts(normalize=True))
  print("\n")

60199014607    0.005556
60166578829    0.005556
60127993102    0.005556
60195194572    0.005556
60197716231    0.005556
                 ...   
60193884620    0.005556
60192525201    0.005556
60123952291    0.005556
60173212331    0.005556
60145224748    0.005556
Name: phonenum, Length: 180, dtype: float64


Johor              0.166667
Melaka             0.127778
Selangor           0.122222
Kelantan           0.111111
Perak              0.111111
Pulau Pinang       0.083333
Pahang             0.061111
Kedah              0.055556
Kuala Lumpur       0.055556
Terengganu         0.050000
Perlis             0.027778
Negeri Sembilan    0.027778
Name: State, dtype: float64


Unsure    1.0
Name: 11. Are you satisfied with the current State Government’s performance?, dtype: float64


Series([], Name: 12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?, dtype: float64)


Series([], Name: 13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legac

In [222]:
# Export as .xlsx file
# set5.to_excel('ivr_pkr2023_set5_cleaned.xlsx', index=False)

### SET 6

In [223]:
set6.head(3)

,0,1,2,3,4,5,Region,Set,6,7,8,9,10,11,12
598,60148067252,FlowNo_2=1,FlowNo_3=1,FlowNo_4=6,FlowNo_5=2,FlowNo_6=1,K1,SET 6,FlowNo_7=1,NaN,NaN,NaN,NaN,NaN,NaN
1431,60125436066,FlowNo_2=2,FlowNo_3=1,FlowNo_4=3,FlowNo_5=2,FlowNo_6=3,K1,SET 6,FlowNo_7=1,NaN,NaN,NaN,NaN,NaN,NaN
1453,60199191617,FlowNo_2=1,FlowNo_3=1,FlowNo_4=3,FlowNo_5=4,FlowNo_6=1,K1,SET 6,FlowNo_7=1,NaN,NaN,NaN,NaN,NaN,NaN


In [224]:
# Map responses accordingly (But, for state & ethnic questions have to be mapped conditionally, based on the Region):

# 14. Do you plan to vote in the upcoming state elections?
q14_mapping = {'FlowNo_3=1': 'Yes',
              'FlowNo_3=2': 'No',
              'FlowNo_3=3': 'Unsure'}

# 15. What are the two most important factors when choosing which party to vote for in the state elections
# q15a
q15a_mapping = {'FlowNo_4=1': "Party’s ability to increase economic opportunities in the state",
              'FlowNo_4=2': "Party’s ability to safeguard my ethno-religious interests",
              'FlowNo_4=3': "Party’s ability to tackle corruption",
              'FlowNo_4=4': "Party’s ability to form a stable government",
              'FlowNo_4=5': "Party’s choice of Menteri Besar or Chief Minister",
              'FlowNo_4=6': "Party is part of Federal Government"}

# q15b
q15b_mapping = {'FlowNo_5=1': "Party’s ability to increase economic opportunities in the state",
              'FlowNo_5=2': "Party’s ability to safeguard my ethno-religious interests",
              'FlowNo_5=3': "Party’s ability to tackle corruption",
              'FlowNo_5=4': "Party’s ability to form a stable government",
              'FlowNo_5=5': "Party’s choice of Menteri Besar or Chief Minister",
              'FlowNo_5=6': "Party is part of Federal Government"}

gender = {'FlowNo_7=1': 'L ',
          'FlowNo_7=2': 'P '}

# Total CR
print(f"Total CR for Set 6: {set6.shape[0]}")

# Select relevant columns only
set6.dropna(axis='columns', inplace=True)
  
# Rename columns
set6.columns = ['phonenum',
                'State',
                '14. Do you plan to vote in the upcoming state elections?',
                'q15a',
                'q15b',
                'ethgroup',
                'Region',
                'Set',
                'gender']

# Map response value
set6['14. Do you plan to vote in the upcoming state elections?'].replace(q14_mapping, inplace=True)

set6['q15a'].replace(q15a_mapping, inplace=True)

set6['q15b'].replace(q15b_mapping, inplace=True)

set6['15. What are the two most important factors when choosing which party to vote for in the state elections?'] = set6['q15a'] + ', ' + set6['q15b']

set6['gender'].replace(gender, inplace=True)

# drop the `q15b` and `q15a` columns
set6 = set6[['phonenum', 
             'State',
             '14. Do you plan to vote in the upcoming state elections?',
             '15. What are the two most important factors when choosing which party to vote for in the state elections?',
             'ethgroup', 'Region', 'Set', 'gender']]

set6.head(3)

Total CR for Set 6: 423


,phonenum,State,14. Do you plan to vote in the upcoming state elections?,15. What are the two most important factors when choosing which party to vote for in the state elections?,ethgroup,Region,Set,gender
598,60148067252,FlowNo_2=1,Yes,"Party is part of Federal Government, Party’s a...",FlowNo_6=1,K1,SET 6,L
1431,60125436066,FlowNo_2=2,Yes,"Party’s ability to tackle corruption, Party’s ...",FlowNo_6=3,K1,SET 6,L
1453,60199191617,FlowNo_2=1,Yes,"Party’s ability to tackle corruption, Party’s ...",FlowNo_6=1,K1,SET 6,L


In [225]:
# Assign State values according to Region
set6.loc[set6['Region'] == ' K1', 'State'] = set6.loc[set6['Region'] == ' K1', 'State'].replace(kluster1)

set6.loc[set6['Region'] == ' K2', 'State'] = set6.loc[set6['Region'] == ' K2', 'State'].replace(kluster2)

set6.loc[set6['Region'] == ' K3', 'State'] = set6.loc[set6['Region'] == ' K3', 'State'].replace(kluster3)

set6.loc[set6['Region'] == ' K4', 'State'] = set6.loc[set6['Region'] == ' K4', 'State'].replace(kluster4)

set6.loc[set6['Region'] == ' K5', 'State'] = set6.loc[set6['Region'] == ' K5', 'State'].replace(kluster5)

In [226]:
# Assign Ethnic values according to Region

# Extract only the key press value in ethgroup column
set6['ethgroup_key'] = set6['ethgroup'].str[-1]

set6.loc[set6['Region'] != ' K5', 'ethgroup'] = set6.loc[set6['Region'] != ' K5', 'ethgroup_key'].replace(peninsular_eth)

set6.loc[set6['Region'] == ' K5', 'ethgroup'] = set6.loc[set6['Region'] == ' K5', 'ethgroup_key'].replace(east_eth)

set6.drop(['ethgroup_key'], axis='columns', inplace=True)

In [227]:
# drop rows where all columns contain the string pattern 'FlowNo_'
set6 = set6[~set6.apply(lambda x: x.str.contains('FlowNo_')).any(axis=1)]

In [228]:
set6.head(3)

,phonenum,State,14. Do you plan to vote in the upcoming state elections?,15. What are the two most important factors when choosing which party to vote for in the state elections?,ethgroup,Region,Set,gender
598,60148067252,Kelantan,Yes,"Party is part of Federal Government, Party’s a...",MELAYU,K1,SET 6,L
1431,60125436066,Terengganu,Yes,"Party’s ability to tackle corruption, Party’s ...",INDIA,K1,SET 6,L
1453,60199191617,Kelantan,Yes,"Party’s ability to tackle corruption, Party’s ...",MELAYU,K1,SET 6,L


In [229]:
# final check
# for x in set6.columns:
#   print(set6[x].value_counts(normalize=True))
#   print("\n")

In [230]:
# Export as .xlsx file
# set6.to_excel('ivr_pkr2023_set6_cleaned.xlsx', index=False)

### SET 7

In [231]:
set7.head(3)

,0,1,2,3,4,5,Region,Set,6,7,8,9,10,11,12
203,60134856332,FlowNo_2=1,FlowNo_3=1,FlowNo_4=2,FlowNo_5=1,FlowNo_6=1,K2,SET 7,FlowNo_7=2,NaN,NaN,NaN,NaN,NaN,NaN
276,60194585551,FlowNo_2=1,FlowNo_3=1,FlowNo_4=1,FlowNo_5=1,FlowNo_6=1,K2,SET 7,FlowNo_7=1,NaN,NaN,NaN,NaN,NaN,NaN
533,60194192116,FlowNo_2=2,FlowNo_3=2,FlowNo_4=1,FlowNo_5=3,FlowNo_6=3,K2,SET 7,FlowNo_7=1,NaN,NaN,NaN,NaN,NaN,NaN


In [232]:
# Map responses accordingly

# 16. From the following options, who are you most likely to vote for? [PN v. PH]
q16_mapping = {'FlowNo_3=1': 'PN',
              'FlowNo_3=2': 'PH'} 

# 17. From the following options, who are you most likely to vote for? [PN v. BN]
q17_mapping = {'FlowNo_4=1': 'PN',
              'FlowNo_4=2': 'BN'}

# 18. From the following options, who are you most likely to vote for? [3 WAY]
q18_mapping = {'FlowNo_5=1': 'PN',
              'FlowNo_5=2': 'BN',
              'FlowNo_5=3': 'PH'}

gender = {'FlowNo_7=1': 'L ',
          'FlowNo_7=2': 'P '}

# Total CR
print(f"Total CR for Set 7: {set7.shape[0]}")

# Select relevant columns only
set7.dropna(axis='columns', inplace=True)
  
# Rename columns
set7.columns = ['phonenum',
                'State',
                '16. From the following options, who are you most likely to vote for? [PN v. PH]',
                '17. From the following options, who are you most likely to vote for? [PN v. BN]',
                '18. From the following options, who are you most likely to vote for? [3 WAY]',
                'ethgroup',
                'Region',
                'Set',
                'gender',]

# Map response value
set7['16. From the following options, who are you most likely to vote for? [PN v. PH]'].replace(q16_mapping, inplace=True)

set7['17. From the following options, who are you most likely to vote for? [PN v. BN]'].replace(q17_mapping, inplace=True)

set7['18. From the following options, who are you most likely to vote for? [3 WAY]'].replace(q18_mapping, inplace=True)

set7['gender'].replace(gender, inplace=True)

set7.head(3)

Total CR for Set 7: 356


,phonenum,State,"16. From the following options, who are you most likely to vote for? [PN v. PH]","17. From the following options, who are you most likely to vote for? [PN v. BN]","18. From the following options, who are you most likely to vote for? [3 WAY]",ethgroup,Region,Set,gender
203,60134856332,FlowNo_2=1,PN,BN,PN,FlowNo_6=1,K2,SET 7,P
276,60194585551,FlowNo_2=1,PN,PN,PN,FlowNo_6=1,K2,SET 7,L
533,60194192116,FlowNo_2=2,PH,PN,PH,FlowNo_6=3,K2,SET 7,L


In [233]:
# Assign State values according to Region
set7.loc[set7['Region'] == ' K1', 'State'] = set7.loc[set7['Region'] == ' K1', 'State'].replace(kluster1)

set7.loc[set7['Region'] == ' K2', 'State'] = set7.loc[set7['Region'] == ' K2', 'State'].replace(kluster2)

set7.loc[set7['Region'] == ' K3', 'State'] = set7.loc[set7['Region'] == ' K3', 'State'].replace(kluster3)

set7.loc[set7['Region'] == ' K4', 'State'] = set7.loc[set7['Region'] == ' K4', 'State'].replace(kluster4)

set7.loc[set7['Region'] == ' K5', 'State'] = set7.loc[set7['Region'] == ' K5', 'State'].replace(kluster5)

In [234]:
# Assign Ethnic values according to Region

# Extract only the key press value in ethgroup column
set7['ethgroup_key'] = set7['ethgroup'].str[-1]

set7.loc[set7['Region'] != ' K5', 'ethgroup'] = set7.loc[set7['Region'] != ' K5', 'ethgroup_key'].replace(peninsular_eth)

set7.loc[set7['Region'] == ' K5', 'ethgroup'] = set7.loc[set7['Region'] == ' K5', 'ethgroup_key'].replace(east_eth)

set7.drop(['ethgroup_key'], axis='columns', inplace=True)

In [235]:
# drop rows where all columns still contain the string pattern 'FlowNo_'
set7 = set7[~set7.apply(lambda x: x.str.contains('FlowNo_')).any(axis=1)]

In [236]:
# final check
# for x in set7.columns:
#   print(set7[x].value_counts(normalize=True))
#   print("\n")

In [237]:
set7.head(3)

,phonenum,State,"16. From the following options, who are you most likely to vote for? [PN v. PH]","17. From the following options, who are you most likely to vote for? [PN v. BN]","18. From the following options, who are you most likely to vote for? [3 WAY]",ethgroup,Region,Set,gender
203,60134856332,Kedah,PN,BN,PN,MELAYU,K2,SET 7,P
276,60194585551,Kedah,PN,PN,PN,MELAYU,K2,SET 7,L
533,60194192116,Pulau Pinang,PH,PN,PH,INDIA,K2,SET 7,L


## Combine IVR Results
for matching purposes in Databricks

In [238]:
ivr_merge = pd.concat([set1, set2, set3, set4, set5, set6, set7], axis='index')

# rearrange columns
ivr_merge = ivr_merge[['phonenum',
                '1. Which of the following issues is most important to you?',
                '2. Are you satisfied with the current economic situation?',
                '4a. Anwar Ibrahim',
                '4b. Muhyiddin Yassin',
                '4c. Zahid Hamidi',
                '4d. Hadi Awang',
                '4e. Fadhilah Yusof',
                '4f. Rafizi Ramli',
                '4g. Anthony Loke',
                '4h. Mohamad Hasan',
                '4i. Mohamad Sabu',
                '5. Are you satisfied with the performance of the current government?',
                "6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?",
                "7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?",
                '8. Are you satisfied with the performance of the Prime Minister?',
                '9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?',
                "11. Are you satisfied with the current State Government’s performance?",
                "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?",
                "13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?",
                '14. Do you plan to vote in the upcoming state elections?',
                '15. What are the two most important factors when choosing which party to vote for in the state elections?',
                '16. From the following options, who are you most likely to vote for? [PN v. PH]',
                '17. From the following options, who are you most likely to vote for? [PN v. BN]',
                '18. From the following options, who are you most likely to vote for? [3 WAY]',
                'ethgroup',
                'gender',
                'Set',
                'State',
                'Region']]

In [239]:
ivr_merge.to_excel('ivr_may_pkr2023_all_sets_cleaned_v{}.xlsx'.format(formatted_date.replace('/', '')), index=False)

## CR count per State

In [240]:
ivr_merge['State'].value_counts()

Kelantan           425
Selangor           415
Kedah              391
Johor              365
Pulau Pinang       296
Terengganu         240
Melaka             229
Perak              223
Pahang             194
Kuala Lumpur       158
Sabah              131
Negeri Sembilan    103
Perlis              83
Sarawak             76
Lain-lain           40
Name: State, dtype: int64

## Check for any null values in demographic columns

In [241]:
ivr_merge[['gender', 'ethgroup', 'State', 'Region']].isnull().sum()

gender      0
ethgroup    0
State       0
Region      0
dtype: int64

## Progress Check: Raw CR count per Set & Region

In [242]:
# by Set
print('CR count by Set:')
print('----------------')
ivr_merge['Set'].value_counts()

CR count by Set:
----------------


SET 1    735
SET 3    682
SET 4    608
SET 6    423
SET 2    385
SET 7    356
SET 5    180
Name: Set, dtype: int64

In [243]:
# by Region
print('CR count by Region:')
print('-------------------')
ivr_merge['Region'].value_counts()

CR count by Region:
-------------------


 K3    939
 K1    859
 K2    770
 K4    594
 K5    207
Name: Region, dtype: int64

In [244]:
sets = [set1, set2, set3, set4,
        set5, set6, set7]

# track State progress by Set.
df_list = []
for i, j in enumerate(sets):
  value_counts = pd.value_counts(j['State'])
  df_list.append(pd.DataFrame({'State': value_counts.index, f'Set {i+1}': value_counts}).set_index('State'))

df_combined = pd.concat(df_list, axis=1)

df_combined.to_excel('ivr_progress_check_v{}.xlsx'.format(formatted_date.replace('/', '')))

print('                        State CR count by Set                          ')
print('-----------------------------------------------------------------------')
df_combined

                        State CR count by Set                          
-----------------------------------------------------------------------


,Set 1,Set 2,Set 3,Set 4,Set 5,Set 6,Set 7
State,,,,,,,
Kedah,97.0,18,100.0,73.0,10.0,55.0,38.0
Kelantan,91.0,52,81.0,72.0,20.0,53.0,56.0
Johor,68.0,43,68.0,75.0,30.0,44.0,37.0
Selangor,68.0,47,82.0,93.0,22.0,52.0,51.0
Sabah,67.0,27,37.0,NaN,NaN,NaN,NaN
Pulau Pinang,62.0,13,65.0,67.0,15.0,41.0,33.0
Terengganu,48.0,17,57.0,45.0,9.0,35.0,29.0
Melaka,44.0,25,31.0,42.0,23.0,38.0,26.0
Sarawak,41.0,8,27.0,NaN,NaN,NaN,NaN


## Read matched results

`ivr_pkr2023_all_sets_cleaned.xlsx` (or `ivr_merge` df) has gone thru phone no. matching in databricks (using Bluebox) to get other demographic information, and saved as `ivr_pkr2023_all_sets_matched.csv`.

In [325]:
ivr_matched = pd.read_csv('ivr_may_pkr2023_all_sets_matched_v{}.csv'.format(formatted_date.replace('/', '')))

# Drop irrelevant columns
ivr_matched.drop(['age_2023', 'NamaParlimen', 'gender_validation','ethnic_validation', 'gender_ethnic_matched',
                  'state_validation', 'ethgroup_temp', 'pred_incomegroup'],
                 axis='columns',
                 inplace=True)

In [326]:
# reorder columns
ivr_matched = ivr_matched[['phonenum',
                '1. Which of the following issues is most important to you?',
                '2. Are you satisfied with the current economic situation?',
                '4a. Anwar Ibrahim',
                '4b. Muhyiddin Yassin',
                '4c. Zahid Hamidi',
                '4d. Hadi Awang',
                '4e. Fadhilah Yusof',
                '4f. Rafizi Ramli',
                '4g. Anthony Loke',
                '4h. Mohamad Hasan',
                '4i. Mohamad Sabu',
                '5. Are you satisfied with the performance of the current government?',
                "6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?",
                "7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?",
                '8. Are you satisfied with the performance of the Prime Minister?',
                '9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?',
                "11. Are you satisfied with the current State Government’s performance?",
                "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?",
                "13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?",
                '14. Do you plan to vote in the upcoming state elections?',
                '15. What are the two most important factors when choosing which party to vote for in the state elections?',
                '16. From the following options, who are you most likely to vote for? [PN v. PH]',
                '17. From the following options, who are you most likely to vote for? [PN v. BN]',
                '18. From the following options, who are you most likely to vote for? [3 WAY]',
                'ethgroup',
                'agegroup',
                'gender',
                'urbanity',
                'State',
                'Region',
                'Set']]

In [327]:
ivr_matched.shape

(2693, 32)

In [328]:
# Check if all changes has been reflected
for col in ivr_matched.columns:
  print(ivr_matched[col].value_counts(normalize=True))
  print("\n")

60134251483    0.000371
60197204622    0.000371
60199885103    0.000371
60165758020    0.000371
60199585232    0.000371
                 ...   
60145124215    0.000371
60145289598    0.000371
60137965110    0.000371
60133981940    0.000371
60172947254    0.000371
Name: phonenum, Length: 2693, dtype: float64


Cost of living and job opportunities    0.621716
Strength of national economy            0.120841
Corruption                              0.108581
Political stability                     0.087566
Quality of national education           0.061296
Name: 1. Which of the following issues is most important to you?, dtype: float64


No        0.588441
Unsure    0.227671
Yes       0.183888
Name: 2. Are you satisfied with the current economic situation?, dtype: float64


1) Very negative    0.329932
5) Very positive    0.326531
3) Neutral          0.176871
2) Negative         0.095238
4) Positive         0.071429
Name: 4a. Anwar Ibrahim, dtype: float64


1) Very negative    0.357143
5) Ver

In [329]:
set1_matched = ivr_matched.loc[ivr_matched['Set'] == 'SET 1']

set2_matched = ivr_matched.loc[ivr_matched['Set'] == 'SET 2']

set3_matched = ivr_matched.loc[ivr_matched['Set'] == 'SET 3']

set4_matched = ivr_matched.loc[ivr_matched['Set'] == 'SET 4']

set5_matched = ivr_matched.loc[ivr_matched['Set'] == 'SET 5']

set6_matched = ivr_matched.loc[ivr_matched['Set'] == 'SET 6']

set7_matched = ivr_matched.loc[ivr_matched['Set'] == 'SET 7']

set8_matched = ivr_matched.loc[ivr_matched['Set'] == 'SET 8']

## Standardize IVR responses

In [ ]:
# ivr_matched = pd.read_excel('ivr_april_pkr2023_all_sets_cleaned_v{}.xlsx'.format(formatted_date.replace('/', '')))

In [330]:
# Reassign values in Region column to match with Live results
clust_region = {' K1': 'Northeast',
                ' K2': 'Northwest',
                ' K3': 'Central',
                ' K4': 'South',
                ' K5': 'East'}

ivr_matched['Region'].replace(clust_region, inplace=True)

# Reassign gender values
gender_reassign = {'L ': 'Male',
                   'P ': 'Female'}

ivr_matched['gender'].replace(gender_reassign, inplace=True)

# Reassign ethgroup values. BUMIPUTERA has to be reassign with original value (Muslim/Non-muslim)
ethnic_reassign = {'MELAYU': 'Malay',
                   'CINA': 'Chinese',
                   'INDIA': 'Indian',
                   'MUSLIM BUMIPUTERA': 'Muslim Bumiputera',
                   'NON-MUSLIM BUMIPUTERA': 'Non-Muslim Bumiputera',
                   'LAIN-LAIN': 'Others'}

ivr_matched['ethgroup'].replace(ethnic_reassign, inplace=True)

# Reassign urbanity values
urban_reassign = {'RURAL': 'Rural',
                  'URBAN': 'Urban',
                  'SEMI-URBAN': 'Urban'}

ivr_matched['urbanity'].replace(urban_reassign, inplace=True)

In [331]:
ivr_matched.head(3)

,phonenum,1. Which of the following issues is most important to you?,2. Are you satisfied with the current economic situation?,4a. Anwar Ibrahim,4b. Muhyiddin Yassin,4c. Zahid Hamidi,4d. Hadi Awang,4e. Fadhilah Yusof,4f. Rafizi Ramli,4g. Anthony Loke,...,"16. From the following options, who are you most likely to vote for? [PN v. PH]","17. From the following options, who are you most likely to vote for? [PN v. BN]","18. From the following options, who are you most likely to vote for? [3 WAY]",ethgroup,agegroup,gender,urbanity,State,Region,Set
0,60134251483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PN,BN,PN,Malay,65+,Male,Rural,Kedah,Northwest,SET 7
1,60124336417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Malay,65+,Female,Urban,Pulau Pinang,Northwest,SET 3
2,60195064437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Malay,65+,Male,Urban,Kuala Lumpur,Central,SET 3


# Live

## Import Live survey results from Google Sheet

In [332]:
import numpy as np
import pandas as pd

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# import response data from 'Clean CR' tab from the following google sheet
worksheet1 = gc.open_by_url('https://docs.google.com/spreadsheets/d/14g0JThrna1gvGTeK52ViTuIcn1K1q2aH1o2p03TreY0/edit?resourcekey#gid=763602281').worksheet('Clean CR')

# get_all_values gives a list of rows
raw = worksheet1.get_all_values()

# Convert to a DataFrame and render
raw = pd.DataFrame(raw[1:], columns = raw[0])

In [333]:
# Replace blank cells as np.nan, else they won't be recognized as null values
live = raw.replace("", np.NaN)

live.head(3)

,Phone Number (Copy from ARIA),What state is your registered address?,Call Status (1),1. Which of the following issues is most important to you?,2. Are you satisfied with the current economic situation?,"3. Where do you get most of your updates about current issues from?\nTIDAK PERLU BACAKAN PILIHAN JAWAPAN, TANDAKAN MENGIKUT JAWAPAN RESPONDEN","4. I will list several government and opposition politicians. Please state whether you feel VERY POSITIVE, POSITIVE, NEUTRAL, NEGATIVE, or VERY NEGATIVE about these leaders. [Anwar Ibrahim]","4. I will list several government and opposition politicians. Please state whether you feel VERY POSITIVE, POSITIVE, NEUTRAL, NEGATIVE, or VERY NEGATIVE about these leaders. [Muhyiddin Yassin]","4. I will list several government and opposition politicians. Please state whether you feel VERY POSITIVE, POSITIVE, NEUTRAL, NEGATIVE, or VERY NEGATIVE about these leaders. [Zahid Hamidi]","4. I will list several government and opposition politicians. Please state whether you feel VERY POSITIVE, POSITIVE, NEUTRAL, NEGATIVE, or VERY NEGATIVE about these leaders. [Hadi Awang]",...,15. What are the two most important factors when choosing which party to vote for in the state elections,"16. Daripada pilihan berikut, siapa yang mungkin anda undi? From the following options, who are you most likely to vote for? [PN v. PH]","17. Daripada pilihan berikut, siapa yang mungkin anda undi? From the following options, who are you most likely to vote for? [PN v. BN]","18. From the following options, who are you most likely to vote for? [3 WAY]",19. What is your age?,20. Ethnicity,21. Gender,22. What is your highest level of education?,Did you lived in...,Call Status (2)
0,60194765373,TERENGGANU,NaN,Cost of living and job opportunities,No,"FACEBOOK, YOUTUBE",NEGATIVE,POSITIVE,NEUTRAL,POSITIVE,...,Party’s ability to increase economic opportuni...,PN,PN,PN,45-54,Melayu,Male,Secondary school,Urban,Call Success
1,60132559239,NEGERI SEMBILAN,NaN,Cost of living and job opportunities,No,ONLINE NEWS,VERY NEGATIVE,POSITIVE,NEGATIVE,POSITIVE,...,Party’s ability to safeguard my ethno-religiou...,OTHERS,OTHERS,UNSURE,25-34,Melayu,Male,Post-secondary,Urban,Call Success
2,60125578462,PERAK,NaN,Cost of living and job opportunities,No,TIKTOK,VERY NEGATIVE,POSITIVE,VERY NEGATIVE,POSITIVE,...,Party’s ability to increase economic opportuni...,PN,PN,PN,45-54,Melayu,Female,Secondary school,Urban,Call Success


## Drop irrelevant columns

In [334]:
live.drop(['Call Status (1)', 'Call Status (2)'], axis='columns', inplace=True)

## Drop incomplete & invalid responses

In [335]:
# For columns without skip logic, we check for incomplete columns
print(f'Before dropping: {len(live)} rows')

live[set(live.columns) - set(["12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?",
                              "13.[IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?",
                              '6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?',
                              '7.[IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?',])]\
                              .dropna(axis='index', inplace=True)

print(f'After dropping: {len(live)} rows')

Before dropping: 685 rows
After dropping: 685 rows


In [336]:
# Manual inspection shows several entries in q6. & q7. violate skip logic. We'll manually remove this. 

# If q5. == 'Yes', then q7. ought to be empty
live.loc[live['5. Are you satisfied with the performance of the current government?'] == 'Yes',
              "7.[IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?"] = live.loc[live['5. Are you satisfied with the performance of the current government?'] == 'Yes',
              "7.[IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?"].replace("", inplace=True)

# If q5. == 'No', then q6. ought to be empty
live.loc[live['5. Are you satisfied with the performance of the current government?'] == 'No',
              "6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?"] = live.loc[live['5. Are you satisfied with the performance of the current government?'] == 'No',
              "6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?"].replace("", inplace=True)

# If q5. == 'Unsure', then q6. & q7. ought to be empty
live.loc[live['5. Are you satisfied with the performance of the current government?'] == 'UNSURE',
              ["6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?",
               "7.[IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?"]] = live.loc[live['5. Are you satisfied with the performance of the current government?'] == 'Unsure',
              ["6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?",
               "7.[IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?"]].replace("", inplace=True)

In [337]:
# Just in case any entries violate skip logic. We'll manually remove this. 

# If q11. == 'Yes', then q13. ought to be empty
live.loc[live["11. Are you satisfied with the current State Government’s performance?"] == 'Yes',
              "13.[IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?"] = live.loc[live["11. Are you satisfied with the current State Government’s performance?"] == 'Yes',
              "13.[IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?"].replace("", inplace=True)

# If q11. == 'No', then q12. ought to be empty
live.loc[live["11. Are you satisfied with the current State Government’s performance?"] == 'No',
              "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?"] = live.loc[live["11. Are you satisfied with the current State Government’s performance?"] == 'No',
              "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?"].replace("", inplace=True)

# If q11. == 'Unsure', then q12. & q13. ought to be empty
live.loc[live["11. Are you satisfied with the current State Government’s performance?"] == 'UNSURE',
              ["13.[IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?",
              "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?"]] = live.loc[live["11. Are you satisfied with the current State Government’s performance?"] == 'Unsure',
              ["13.[IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?",
              "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?"]].replace("", inplace=True)

In [338]:
# Check for completeness for q6. and q7.
live = live[((live["5. Are you satisfied with the performance of the current government?"] == 'Yes') & \
             (~live["6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?"].isnull())) | \
              ((live["5. Are you satisfied with the performance of the current government?"] == 'No') & \
             (~live["7.[IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?"].isnull())) | \
              (live["5. Are you satisfied with the performance of the current government?"] == 'UNSURE')]

# Check for completeness for q12. and q13.
live = live[((live["11. Are you satisfied with the current State Government’s performance?"] == 'Yes') & \
             (~live["12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?"].isnull())) | \
              ((live["11. Are you satisfied with the current State Government’s performance?"] == 'No') & \
             (~live["13.[IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?"].isnull())) | \
              (live["11. Are you satisfied with the current State Government’s performance?"] == 'UNSURE')]

print(f'After removing invalid entries and completeness checks: {len(live)} rows')

After removing invalid entries and completeness checks: 684 rows


## Rename & reorder columns

In [339]:
rename_col = ['phonenum', 
              'State',
              '1. Which of the following issues is most important to you?',
              '2. Are you satisfied with the current economic situation?',
              '3. Where do you get most of your updates about current issues from?',
              '4a. Anwar Ibrahim',
              '4b. Muhyiddin Yassin',
              '4c. Zahid Hamidi',
              '4d. Hadi Awang',
              '4e. Fadhilah Yusof',
              '4f. Rafizi Ramli',
              '4g. Anthony Loke',
              '4h. Mohamad Hasan',
              '4i. Mohamad Sabu',
              '4j. Saifuddin Nasution Ismail',
              '5. Are you satisfied with the performance of the current government?',
              "6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?",
              "7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?",
              '8. Are you satisfied with the performance of the Prime Minister?',
              '9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?',
              "10. If you are willing to answer, which party did you vote for in GE15?",
              "11. Are you satisfied with the current State Government’s performance?",
              "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?",
              "13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?",
              '14. Do you plan to vote in the upcoming state elections?',
              "15. What are the two most important factors when choosing which party to vote for in the state elections?",
              '16. From the following options, who are you most likely to vote for? [PN v. PH]',
              '17. From the following options, who are you most likely to vote for? [PN v. BN]',
              '18. From the following options, who are you most likely to vote for? [3 WAY]',
              'agegroup', 'ethgroup', 'gender', 'edulevel', 'urbanity']

reorder_col = ['phonenum', 
              'State',
              '1. Which of the following issues is most important to you?',
              '2. Are you satisfied with the current economic situation?',
              '3. Where do you get most of your updates about current issues from?',
              '4a. Anwar Ibrahim',
              '4b. Muhyiddin Yassin',
              '4c. Zahid Hamidi',
              '4d. Hadi Awang',
              '4e. Fadhilah Yusof',
              '4f. Rafizi Ramli',
              '4g. Anthony Loke',
              '4h. Mohamad Hasan',
              '4i. Mohamad Sabu',
              '4j. Saifuddin Nasution Ismail',
              '5. Are you satisfied with the performance of the current government?',
              "6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?",
              "7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?",
              '8. Are you satisfied with the performance of the Prime Minister?',
              '9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?',
              "10. If you are willing to answer, which party did you vote for in GE15?",
              "11. Are you satisfied with the current State Government’s performance?",
              "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?",
              "13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?",
              '14. Do you plan to vote in the upcoming state elections?',
              "15. What are the two most important factors when choosing which party to vote for in the state elections?",
              '16. From the following options, who are you most likely to vote for? [PN v. PH]',
              '17. From the following options, who are you most likely to vote for? [PN v. BN]',
              '18. From the following options, who are you most likely to vote for? [3 WAY]',
              'agegroup', 'ethgroup', 'gender', 'edulevel', 'urbanity']

In [340]:
# Rename columns
live.columns = rename_col

# Reorder columns ('State' goes last)
live = live[reorder_col]

In [341]:
# Check if all changes has been reflected
for col in live.columns:
  print(live[col].value_counts(normalize=True))
  print("\n")

60192459673     0.002924
60175172741     0.002924
60126147048     0.002924
60135205515     0.002924
601129504192    0.002924
                  ...   
60176507858     0.001462
60199009807     0.001462
60122607625     0.001462
60199586775     0.001462
60199649173     0.001462
Name: phonenum, Length: 669, dtype: float64


SELANGOR           0.230994
JOHOR              0.182749
NEGERI SEMBILAN    0.084795
PERAK              0.080409
KUALA LUMPUR       0.062865
KELANTAN           0.058480
KEDAH              0.058480
TERENGGANU         0.054094
PAHANG             0.049708
SABAH              0.045322
MELAKA             0.039474
PULAU PINANG       0.027778
SARAWAK            0.016082
PERLIS             0.007310
LABUAN             0.001462
Name: State, dtype: float64


Cost of living and job opportunities    0.502924
Strength of national economy            0.191520
Corruption                              0.111111
Political stability                     0.099415
Quality of national education    

## Standardize responses

In [342]:
dynamo = {
    'Neglect of Islam and Malay-Muslim institution': 'Neglect of Islam and Malay-Muslim institutions',
    'Neglect of Islam and Malay-Muslim institutionson 1': 'Neglect of Islam and Malay-Muslim institutions',
    'Others         [JANGAN BACA // DON\'T READ]': 'Others',
    'OTHERS': 'Others',
    ' OTHERS': 'Others',
    'Other':'Others',
    'NO': 'No',
    'YES': 'Yes',
    'VERY POSITIVE': '5) Very positive',
    'POSITIVE': '4) Positive',
    'NEUTRAL': '3) Neutral',
    'NEGATIVE': '2) Negative',
    'VERY NEGATIVE': '1) Very negative',
    'NOT RECOGNIZE': '6) Not recognize',
    'UNSURE          [JANGAN BACA // DON\'T READ]': 'Unsure',
    'REFUSE TO ANSWER': 'Refused to answer',
    'UNSURE': 'Unsure',
    'Unity governement': 'Unity Gov.',
    'PN-BN government': 'PN-BN',
    "Refuse to answer           [JANGAN BACA // DON'T READ]": 'Refused to answer',
    'TIDAK MENGUNDI': 'Didn\'t vote',
    'Unsure        [JANGAN BACA // DON\'T READ]': 'Unsure',
    '65 and above': '65+',
    'Luar Bandar': 'Rural',
    'Melayu': 'Malay',
    'India': 'Indian',
    'Cina': 'Chinese',
    'Non-Muslim Bumiputera Sabah-Sarawak': 'Non-Muslim Bumiputera',
    'Muslim Bumiputera Sabah-Sarawak': 'Muslim Bumiputera' 
}

In [343]:
# Function to apply .title() to a string
def title_case(string):
    return string.title()

# Apply .title() to the 'Name' column
live['State'] = live['State'].apply(lambda x: title_case(x))

In [344]:
# Clean responses
live = live.replace(dynamo)

In [345]:
# Check if all changes has been reflected
for col in live.columns:
  print(live[col].value_counts(normalize=True))
  print("\n")

60192459673     0.002924
60175172741     0.002924
60126147048     0.002924
60135205515     0.002924
601129504192    0.002924
                  ...   
60176507858     0.001462
60199009807     0.001462
60122607625     0.001462
60199586775     0.001462
60199649173     0.001462
Name: phonenum, Length: 669, dtype: float64


Selangor           0.230994
Johor              0.182749
Negeri Sembilan    0.084795
Perak              0.080409
Kuala Lumpur       0.062865
Kelantan           0.058480
Kedah              0.058480
Terengganu         0.054094
Pahang             0.049708
Sabah              0.045322
Melaka             0.039474
Pulau Pinang       0.027778
Sarawak            0.016082
Perlis             0.007310
Labuan             0.001462
Name: State, dtype: float64


Cost of living and job opportunities    0.502924
Strength of national economy            0.191520
Corruption                              0.111111
Political stability                     0.099415
Quality of national education    

## Create `Set` column

In [346]:
len(live.columns)

34

In [347]:
live_set = ['Live' for x in range(len(live))]

# Insert `Set` column at the end
# Note: https://www.geeksforgeeks.org/python-pandas-dataframe-insert/
live.insert(len(live.columns), 'Set', live_set)

## Create `Region` column
based on `State` column

In [348]:
# Initialize the Region column
live['Region'] = 'null'

In [349]:
# Set Region that corresponds to State
live.loc[live['State'].isin(['Pahang', 'Terengganu', 'Kelantan']), 'Region'] = 'Northeast'

live.loc[live['State'].isin(['Perlis', 'Kedah', 'Pulau Pinang']), 'Region'] = 'Northwest'

live.loc[live['State'].isin(['Perak', 'Selangor', 'Kuala Lumpur', 'Putrajaya', 'Negeri Sembilan']), 'Region'] = 'Central'

live.loc[live['State'].isin(['Melaka', 'Johor']), 'Region'] = 'South'

live.loc[live['State'].isin(['Sabah', 'Sarawak', 'Labuan']), 'Region'] = 'East'

In [350]:
live.head(3)

,phonenum,State,1. Which of the following issues is most important to you?,2. Are you satisfied with the current economic situation?,3. Where do you get most of your updates about current issues from?,4a. Anwar Ibrahim,4b. Muhyiddin Yassin,4c. Zahid Hamidi,4d. Hadi Awang,4e. Fadhilah Yusof,...,"16. From the following options, who are you most likely to vote for? [PN v. PH]","17. From the following options, who are you most likely to vote for? [PN v. BN]","18. From the following options, who are you most likely to vote for? [3 WAY]",agegroup,ethgroup,gender,edulevel,urbanity,Set,Region
0,60194765373,Terengganu,Cost of living and job opportunities,No,"FACEBOOK, YOUTUBE",2) Negative,4) Positive,3) Neutral,4) Positive,4) Positive,...,PN,PN,PN,45-54,Malay,Male,Secondary school,Urban,Live,Northeast
1,60132559239,Negeri Sembilan,Cost of living and job opportunities,No,ONLINE NEWS,1) Very negative,4) Positive,2) Negative,4) Positive,3) Neutral,...,Others,Others,Unsure,25-34,Malay,Male,Post-secondary,Urban,Live,Central
2,60125578462,Perak,Cost of living and job opportunities,No,TIKTOK,1) Very negative,4) Positive,1) Very negative,4) Positive,3) Neutral,...,PN,PN,PN,45-54,Malay,Female,Secondary school,Urban,Live,Central


In [ ]:
# live.to_excel('live_march_pkr2023_cleaned_v2.xlsx', index=False)

# Combine Live & IVR Results
**Note:** 

1. IVR now covers Federal- and State-level question.
2. Also, it seems there's still null values from Live. Will debug later.

In [351]:
live_sub = live[set(live.columns) - set(['edulevel'])]

In [352]:
live_sub.head()

,urbanity,6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?,"9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?",ethgroup,4b. Muhyiddin Yassin,Region,4h. Mohamad Hasan,State,4i. Mohamad Sabu,8. Are you satisfied with the performance of the Prime Minister?,...,4g. Anthony Loke,4c. Zahid Hamidi,4j. Saifuddin Nasution Ismail,gender,5. Are you satisfied with the performance of the current government?,4d. Hadi Awang,"18. From the following options, who are you most likely to vote for? [3 WAY]",11. Are you satisfied with the current State Government’s performance?,phonenum,"17. From the following options, who are you most likely to vote for? [PN v. BN]"
0,Urban,None,PN-BN,Malay,4) Positive,Northeast,3) Neutral,Terengganu,3) Neutral,No,...,3) Neutral,3) Neutral,3) Neutral,Male,No,4) Positive,PN,Yes,60194765373,PN
1,Urban,None,PN-BN,Malay,4) Positive,Central,2) Negative,Negeri Sembilan,3) Neutral,No,...,2) Negative,2) Negative,2) Negative,Male,No,4) Positive,Unsure,Yes,60132559239,Others
2,Urban,None,PN-BN,Malay,4) Positive,Central,3) Neutral,Perak,3) Neutral,No,...,3) Neutral,1) Very negative,1) Very negative,Female,No,4) Positive,PN,No,60125578462,PN
3,Urban,Increased economic opportunities,Unity government,Indian,2) Negative,Central,3) Neutral,Perak,4) Positive,Yes,...,3) Neutral,4) Positive,3) Neutral,Female,Yes,2) Negative,Others,Yes,60135303364,Others
4,Urban,Improvement of federal infrastructure,About the same,Malay,4) Positive,Northeast,3) Neutral,Kelantan,3) Neutral,Yes,...,3) Neutral,2) Negative,3) Neutral,Female,Yes,2) Negative,PH,Unsure,60199095380,PN


In [353]:
# Combine both Live & IVR results
all_results = live_sub.append(ivr_matched)

In [354]:
# remove responses where all_results['State'] == 'Lain-Lain'
all_results = all_results[~(all_results['State'] == 'Lain-lain')]
print(len(all_results)) 

3359


In [355]:
# final sanity check on all columns
for col in all_results.columns:
  print(all_results[col].value_counts(normalize=True))
  print("\n")

Urban    0.805895
Rural    0.194105
Name: urbanity, dtype: float64


Increased economic opportunities                 0.266791
Successfully reducing corruption                 0.238806
Improvement of racial and religious situation    0.195896
Performing leaders or representatives            0.175373
Improvement of federal infrastructure            0.102612
Others                                           0.020522
Name: 6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?, dtype: float64


About the same      0.409628
PN-BN               0.261824
Unity government    0.179054
Unity Gov.          0.149493
Name: 9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?, dtype: float64


Malay                    0.850551
Chinese                  0.079488
Indian                   0.048824
Muslim Bumiputera        0.008634
Non-Muslim Bumiputera    0.008634
Others                   0.003870
Name: ethgroup, dtype:

## Save combined results per Set

### Set 1 + Live

In [356]:
set1_combined = all_results.loc[all_results['Set'].isin(['SET 1', 'Live'])]

set1_combined = set1_combined[[
    'phonenum',
    '1. Which of the following issues is most important to you?',
    '2. Are you satisfied with the current economic situation?',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set']]

set1_combined.shape

(1255, 10)

In [361]:
# final check for null values
# set1_combined.info()

In [357]:
set1_combined.to_excel('ivr_may_pkr2023_set1_combined_v{}.xlsx'.format(formatted_date.replace('/', '')))

### Set 2 + Live

In [358]:
set2_combined = all_results.loc[all_results['Set'].isin(['SET 2', 'Live'])]

set2_combined = set2_combined[[
    'phonenum',
    '4a. Anwar Ibrahim',
    '4b. Muhyiddin Yassin',
    '4c. Zahid Hamidi',
    '4d. Hadi Awang',
    '4e. Fadhilah Yusof',
    '4f. Rafizi Ramli',
    '4g. Anthony Loke',
    '4h. Mohamad Hasan',
    '4i. Mohamad Sabu',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set']]

set2_combined.shape

(960, 17)

In [362]:
# final check for null values
set2_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 960 entries, 0 to 2687
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   phonenum              960 non-null    object
 1   4a. Anwar Ibrahim     960 non-null    object
 2   4b. Muhyiddin Yassin  960 non-null    object
 3   4c. Zahid Hamidi      960 non-null    object
 4   4d. Hadi Awang        960 non-null    object
 5   4e. Fadhilah Yusof    960 non-null    object
 6   4f. Rafizi Ramli      960 non-null    object
 7   4g. Anthony Loke      960 non-null    object
 8   4h. Mohamad Hasan     959 non-null    object
 9   4i. Mohamad Sabu      960 non-null    object
 10  ethgroup              960 non-null    object
 11  agegroup              960 non-null    object
 12  gender                960 non-null    object
 13  urbanity              960 non-null    object
 14  State                 960 non-null    object
 15  Region                960 non-null    o

In [363]:
# we STILL notice some null values *sigh*. Use mode imputation. 
set2_combined['4h. Mohamad Hasan']\
                  .fillna(set2_combined['4h. Mohamad Hasan']\
                  .mode()[0], inplace=True)

In [364]:
set2_combined.to_excel('ivr_may_pkr2023_set2_combined_v{}.xlsx'.format(formatted_date.replace('/', '')))

### Set 3 + Live

In [365]:
set3_combined = all_results.loc[all_results['Set'].isin(['SET 3', 'Live'])]

set3_combined = set3_combined[[
    'phonenum',
    '5. Are you satisfied with the performance of the current government?',
    "6. [IF ‘YES’ TO Q5] What is the current Federal Government’s most positive legacy?",
    "7. [IF ‘NO’ TO Q5] What is the current Federal Government’s most negative legacy?",
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set']]

set3_combined.shape

(1227, 11)

In [368]:
# final check for null values
# set3_combined.info()

In [367]:
set3_combined.to_excel('ivr_may_pkr2023_set3_combined_v{}.xlsx'.format(formatted_date.replace('/', '')))

### Set 4 + Live

In [369]:
set4_combined = all_results.loc[all_results['Set'].isin(['SET 4', 'Live'])]

set4_combined = set4_combined[[
    'phonenum',
    '8. Are you satisfied with the performance of the Prime Minister?',
    '9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set']]

set4_combined.shape

(1184, 10)

In [370]:
# final check for null values
# set4_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1184 entries, 0 to 2691
Data columns (total 10 columns):
 #   Column                                                                                                         Non-Null Count  Dtype 
---  ------                                                                                                         --------------  ----- 
 0   phonenum                                                                                                       1184 non-null   object
 1   8. Are you satisfied with the performance of the Prime Minister?                                               1183 non-null   object
 2   9. Between the current unity government and the previous PN-BN government, which are you more satisfied with?  1184 non-null   object
 3   ethgroup                                                                                                       1184 non-null   object
 4   agegroup                                              

In [372]:
# we STILL notice some null values *sigh*. Use mode imputation. 
set4_combined['8. Are you satisfied with the performance of the Prime Minister?']\
                  .fillna(set4_combined['8. Are you satisfied with the performance of the Prime Minister?']\
                  .mode()[0], inplace=True)

In [373]:
set4_combined.to_excel('ivr_may_pkr2023_set4_combined_v{}.xlsx'.format(formatted_date.replace('/', '')))

### Set 5 + Live

In [374]:
set5_combined = all_results.loc[all_results['Set'].isin(['SET 5', 'Live'])]

set5_combined = set5_combined[[
    'phonenum',
    "11. Are you satisfied with the current State Government’s performance?",
    "12. [IF ‘YES’ TO Q11] What is the current state government's most positive legacy?",
    "13. [IF ‘NO’ TO Q11] What is the current state government’s most negative legacy?",
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set']]

set5_combined.shape

(824, 11)

In [376]:
# final check for null values
# set5_combined.info()

In [377]:
set5_combined.to_excel('ivr_may_pkr2023_set5_combined_v{}.xlsx'.format(formatted_date.replace('/', '')))

### Set 6 + Live

In [378]:
set6_combined = all_results.loc[all_results['Set'].isin(['SET 6', 'Live'])]

set6_combined = set6_combined[[
    'phonenum',
    '14. Do you plan to vote in the upcoming state elections?',
    '15. What are the two most important factors when choosing which party to vote for in the state elections?',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set']]

set6_combined.shape

(1043, 10)

In [380]:
# final check for null values
# set6_combined.info()

In [381]:
set6_combined.to_excel('ivr_may_pkr2023_set6_combined_v{}.xlsx'.format(formatted_date.replace('/', '')))

### Set 7 + Live

In [382]:
set7_combined = all_results.loc[all_results['Set'].isin(['SET 7', 'Live'])]

set7_combined = set7_combined[[
    'phonenum',
    '16. From the following options, who are you most likely to vote for? [PN v. PH]',
    '17. From the following options, who are you most likely to vote for? [PN v. BN]',
    '18. From the following options, who are you most likely to vote for? [3 WAY]',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set']]

set7_combined.shape

(970, 11)

In [384]:
# final check for null values
# set7_combined.info()

In [385]:
set7_combined.to_excel('ivr_may_pkr2023_set7_combined_v{}.xlsx'.format(formatted_date.replace('/', '')))

### Set Miscellaneous

In [386]:
setmisc_combined = all_results.loc[all_results['Set'].isin(['Live'])]

setmisc_combined = setmisc_combined[[
    'phonenum',
    '10. If you are willing to answer, which party did you vote for in GE15?',
    '4j. Saifuddin Nasution Ismail',
    '3. Where do you get most of your updates about current issues from?',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set']]

setmisc_combined.shape

(684, 11)

In [387]:
# final check for null values
setmisc_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 684 entries, 0 to 684
Data columns (total 11 columns):
 #   Column                                                                   Non-Null Count  Dtype 
---  ------                                                                   --------------  ----- 
 0   phonenum                                                                 684 non-null    object
 1   10. If you are willing to answer, which party did you vote for in GE15?  684 non-null    object
 2   4j. Saifuddin Nasution Ismail                                            684 non-null    object
 3   3. Where do you get most of your updates about current issues from?      682 non-null    object
 4   ethgroup                                                                 684 non-null    object
 5   agegroup                                                                 684 non-null    object
 6   gender                                                                   684 non-n

In [388]:
# we STILL notice some null values *sigh*. Use mode imputation. 
setmisc_combined['3. Where do you get most of your updates about current issues from?']\
                  .fillna(setmisc_combined['3. Where do you get most of your updates about current issues from?']\
                  .mode()[0], inplace=True)

In [389]:
setmisc_combined.to_excel('ivr_may_pkr2023_setMisc_combined_v{}.xlsx'.format(formatted_date.replace('/', '')))

## Progress Check: Combined CR count per Set & State

In [390]:
# list all combined sets
sets = [set1_combined, set2_combined, set3_combined, set4_combined,
        set5_combined, set6_combined, set7_combined, setmisc_combined]

In [391]:
# by Set
print('CR count by Set (Live + IVR):')
print('-----------------------------')
for i,x in enumerate(sets):
  print(f"Set {i + 1} CR: {x['Set'].count()}")

print("")
print(f"As of: {date.today()}")

CR count by Set (Live + IVR):
-----------------------------
Set 1 CR: 1255
Set 2 CR: 960
Set 3 CR: 1227
Set 4 CR: 1184
Set 5 CR: 824
Set 6 CR: 1043
Set 7 CR: 970
Set 8 CR: 684

As of: 2023-06-01


In [392]:
PRN = ['Selangor', 'Pulau Pinang', 'Kelantan', 'Terengganu', 'Negeri Sembilan', 'Kedah']

# we'll add CR live to track progress.
df_list = []
for i, j in enumerate(sets):
  j['State_reg'] = j['State'].apply(lambda r: 'Others' if r not in PRN else r)
  value_counts = pd.value_counts(j['State_reg'])
  df_list.append(pd.DataFrame({'State_reg': value_counts.index, f'Set {i+1}': value_counts}).set_index('State_reg'))

df_combined = pd.concat(df_list, axis=1)

df_combined.to_excel('progress_check_v{}.xlsx'.format(formatted_date.replace('/', '')))

print(f"As of: {date.today()}")
df_combined

As of: 2023-06-01


,Set 1,Set 2,Set 3,Set 4,Set 5,Set 6,Set 7,Set 8
State_reg,,,,,,,,
Others,570,477,527,530,410,477,441,332
Selangor,216,198,227,232,174,203,197,158
Kedah,126,51,127,103,48,84,72,40
Kelantan,124,85,113,107,56,90,90,40
Terengganu,77,52,90,74,43,65,63,37
Negeri Sembilan,74,69,72,70,62,70,68,58
Pulau Pinang,68,28,71,68,31,54,39,19


In [393]:
# for more granular tracking (by each state, instead of just PRN focus)
df_list = []
for i, j in enumerate(sets):
  value_counts = pd.value_counts(j['State'])
  df_list.append(pd.DataFrame({'State': value_counts.index, f'Set {i+1}': value_counts}).set_index('State'))

df_combined = pd.concat(df_list, axis=1)

df_combined.to_excel('progress_check_v{}.xlsx'.format(formatted_date.replace('/', '')))

print(f"As of: {date.today()}")
df_combined

As of: 2023-06-01


,Set 1,Set 2,Set 3,Set 4,Set 5,Set 6,Set 7,Set 8
State,,,,,,,,
Selangor,216,198,227,232,174,203,197,158
Johor,183,160,179,185,147,166,155,125
Kedah,126,51,127,103,48,84,72,40
Kelantan,124,85,113,107,56,90,90,40
Perak,79,84,84,93,72,76,77,55
Terengganu,77,52,90,74,43,65,63,37
Negeri Sembilan,74,69,72,70,62,70,68,58
Pahang,71,56,68,58,44,63,49,34
Pulau Pinang,68,28,71,68,31,54,39,19


In [ ]:
# sanity check
# for i, x in enumerate(sets):
#   print('Set: {}\n'.format(i+1))
#   print(x['State'].value_counts())
#   print("__________________________")

# Consolidate before Crosstabs

## Combine weighted results (National)
The following files were generated after all individual set's weightage was calculated using R survey package (locally).

*Note: These are NOT the subsetted results for PRN. These are national results.

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

# Adding this code chunk to track progress by date.
today = date.today()
formatted_date = "/" + today.strftime("%d-%m-%Y").replace("-0", "-")
print("Today's date:", formatted_date)

Today's date: /02-5-2023


In [ ]:
weighted_files = [
    'pkr2023-Federal-April-set1-1760CR-weighted.xlsx',
    'pkr2023-Federal-April-set2-1879CR-weighted.xlsx',
    'pkr2023-Federal-April-set3-850CR-weighted.xlsx',
    'pkr2023-Federal-April-set4-1203CR-weighted.xlsx',
    'pkr2023-Federal-April-set5-1007CR-weighted.xlsx',
    'pkr2023-Federal-April-set6-1302CR-weighted.xlsx',
    'pkr2023-Federal-April-set7-1400CR-weighted.xlsx',
    'pkr2023-Federal-April-set8-1068CR-weighted.xlsx'
]

In [ ]:
# Loop thru all weighted files and combine
weighted_list = []

for f in weighted_files:
  df = pd.read_excel(f)
  weighted_list.append(df)

weighted_results = pd.concat(weighted_list, axis='index')

In [ ]:
# Rearrange columns
weighted_results = weighted_results[[
    'phonenum',
    '1. Which of the following issues is most important to you?',
    '2. Are you satisfied with your current economic situation?',
    '4a. BERSATU', '4b. PAS', '4c. UMNO', '4d. PH',
    '5a. Anwar Ibrahim', '5b. Muhyiddin Yassin', '5c. Zahid Hamidi',
    '5d. Hadi Awang', '5e. Fadillah Yusof', '5f. Rafizi Ramli',
    '5g. Anthony Loke', '5h. Mohamad Hasan', '5i. Mohamad Sabu',
    '6. Are you satisfied with your current federal government?',
    '7. Are you satisfied with the performance of the current Prime Minister?',
    '8. Which government are you satisfied with?',
    '9. Which party did you vote for in GE15?',
    '10. Which of the following state-level issues is most important to you?',
    '11. Are you satisfied with the State Government’s performance in addressing this issue? [based on answer to Q10]',
    '12. Are you satisfied with the level of development and economic opportunities in your area?',
    '13. What is the most negative legacy of the current State Government?',
    '14. What is the most positive legacy of the current State Government?',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set',
    'untrimmed_weight',
    'trimmed_weight'
]]

In [ ]:
# Export to be run thru Crosstab Generator
weighted_results.to_excel('pkr2023-april-federal-all-sets-weighted_v{}.xlsx'.format(formatted_date.replace('/', '')), index=False)

## Combine weighted results (PRN States)

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

# Adding this code chunk to track progress by date.
today = date.today()
formatted_date = "/" + today.strftime("%d-%m-%Y").replace("-0", "-")
print("Today's date:", formatted_date)

Today's date: /02-5-2023


In [ ]:
weighted_prn = [
    'pkr2023-PRN6States-April-set-1-997CR-weighted.xlsx',
    'pkr2023-PRN6States-April-set-2-1029CR-weighted.xlsx',
    'pkr2023-PRN6States-April-set-3-469CR-weighted.xlsx',
    'pkr2023-PRN6States-April-set-4-647CR-weighted.xlsx',
    'pkr2023-PRN6States-April-set-5-540CR-weighted.xlsx',
    'pkr2023-PRN6States-April-set-6-714CR-weighted.xlsx',
    'pkr2023-PRN6States-April-set-7-741CR-weighted.xlsx',
    'pkr2023-PRN6States-April-set-8-588CR-weighted.xlsx'
]

In [ ]:
# Loop thru all weighted files and combine
weighted_list_prn = []

for f in weighted_prn:
  df = pd.read_excel(f)
  weighted_list_prn.append(df)

weighted_results_prn = pd.concat(weighted_list_prn, axis='index')

In [ ]:
# Rearrange columns
weighted_results_prn = weighted_results_prn[[
    'phonenum',
    '1. Which of the following issues is most important to you?',
    '2. Are you satisfied with your current economic situation?',
    '4a. BERSATU', '4b. PAS', '4c. UMNO', '4d. PH',
    '5a. Anwar Ibrahim', '5b. Muhyiddin Yassin', '5c. Zahid Hamidi',
    '5d. Hadi Awang', '5e. Fadillah Yusof', '5f. Rafizi Ramli',
    '5g. Anthony Loke', '5h. Mohamad Hasan', '5i. Mohamad Sabu',
    '6. Are you satisfied with your current federal government?',
    '7. Are you satisfied with the performance of the current Prime Minister?',
    '8. Which government are you satisfied with?',
    '9. Which party did you vote for in GE15?',
    '10. Which of the following state-level issues is most important to you?',
    '11. Are you satisfied with the State Government’s performance in addressing this issue? [based on answer to Q10]',
    '12. Are you satisfied with the level of development and economic opportunities in your area?',
    '13. What is the most negative legacy of the current State Government?',
    '14. What is the most positive legacy of the current State Government?',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set',
    'untrimmed_weight',
    'trimmed_weight'
]]

In [ ]:
# Temp. code
likert_mapping = {'Very negative': '1) Very negative',
                  'Negative': '2) Negative',
                  'Neutral': '3) Neutral',
                  'Positive': '4) Positive',
                  'Very positive': '5) Very positive',
                  'Not recognize': '6) Not recognize'}

weighted_results_prn.replace(likert_mapping, inplace=True)

In [ ]:
# Export to be run thru Crosstab Generator
weighted_results_prn.to_excel('pkr2023-PRN-all-sets-weighted_v{}.xlsx'.format(formatted_date.replace('/', '')), index=False)

## Combine weighted results (KELANTAN)

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

# Adding this code chunk to track progress by date.
today = date.today()
formatted_date = "/" + today.strftime("%d-%m-%Y").replace("-0", "-")
print("Today's date:", formatted_date)

Today's date: /03-5-2023


In [ ]:
weighted_prn = [
 'pkr2023-KELANTAN-April-set1-229CR-weighted.xlsx',
 'pkr2023-KELANTAN-April-set2-239CR-weighted.xlsx',
 'pkr2023-KELANTAN-April-set3-98CR-weighted.xlsx',
 'pkr2023-KELANTAN-April-set4-130CR-weighted.xlsx',
 'pkr2023-KELANTAN-April-set5-122CR-weighted.xlsx',
 'pkr2023-KELANTAN-April-set6-164CR-weighted.xlsx',
 'pkr2023-KELANTAN-April-set7-163CR-weighted.xlsx',
 'pkr2023-KELANTAN-April-set8-141CR-weighted.xlsx'
]

In [ ]:
# Loop thru all weighted files and combine
weighted_list_prn = []

for f in weighted_prn:
  df = pd.read_excel(f)
  weighted_list_prn.append(df)

weighted_results_prn = pd.concat(weighted_list_prn, axis='index')

In [ ]:
# Rearrange columns
weighted_results_prn = weighted_results_prn[[
    'phonenum',
    '1. Which of the following issues is most important to you?',
    '2. Are you satisfied with your current economic situation?',
    '4a. BERSATU', '4b. PAS', '4c. UMNO', '4d. PH',
    '5a. Anwar Ibrahim', '5b. Muhyiddin Yassin', '5c. Zahid Hamidi',
    '5d. Hadi Awang', '5e. Fadillah Yusof', '5f. Rafizi Ramli',
    '5g. Anthony Loke', '5h. Mohamad Hasan', '5i. Mohamad Sabu',
    '6. Are you satisfied with your current federal government?',
    '7. Are you satisfied with the performance of the current Prime Minister?',
    '8. Which government are you satisfied with?',
    '9. Which party did you vote for in GE15?',
    '10. Which of the following state-level issues is most important to you?',
    '11. Are you satisfied with the State Government’s performance in addressing this issue? [based on answer to Q10]',
    '12. Are you satisfied with the level of development and economic opportunities in your area?',
    '13. What is the most negative legacy of the current State Government?',
    '14. What is the most positive legacy of the current State Government?',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set',
    'untrimmed_weight',
    'trimmed_weight'
]]

In [ ]:
# Temp. code
likert_mapping = {'Very negative': '1) Very negative',
                  'Negative': '2) Negative',
                  'Neutral': '3) Neutral',
                  'Positive': '4) Positive',
                  'Very positive': '5) Very positive',
                  'Not recognize': '6) Not recognize'}

weighted_results_prn.replace(likert_mapping, inplace=True)

In [ ]:
# Export to be run thru Crosstab Generator
weighted_results_prn.to_excel('pkr2023-PRN-KELANTAN-sets-weighted_v{}.xlsx'.format(formatted_date.replace('/', '')), index=False)

## Combine weighted results (TERENGGANU)

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

# Adding this code chunk to track progress by date.
today = date.today()
formatted_date = "/" + today.strftime("%d-%m-%Y").replace("-0", "-")
print("Today's date:", formatted_date)

Today's date: /03-5-2023


In [ ]:
weighted_prn = [
 'pkr2023-Terengganu-April-set1-138CR-weighted.xlsx',
 'pkr2023-Terengganu-April-set2-140CR-weighted.xlsx',
 'pkr2023-Terengganu-April-set3-51CR-weighted.xlsx',
 'pkr2023-Terengganu-April-set4-72CR-weighted.xlsx',
 'pkr2023-Terengganu-April-set5-63CR-weighted.xlsx',
 'pkr2023-Terengganu-April-set6-89CR-weighted.xlsx',
 'pkr2023-Terengganu-April-set7-103CR-weighted.xlsx',
 'pkr2023-Terengganu-April-set8-68CR-weighted.xlsx'
]

In [ ]:
# Loop thru all weighted files and combine
weighted_list_prn = []

for f in weighted_prn:
  df = pd.read_excel(f)
  weighted_list_prn.append(df)

weighted_results_prn = pd.concat(weighted_list_prn, axis='index')

In [ ]:
# Rearrange columns
weighted_results_prn = weighted_results_prn[[
    'phonenum',
    '1. Which of the following issues is most important to you?',
    '2. Are you satisfied with your current economic situation?',
    '4a. BERSATU', '4b. PAS', '4c. UMNO', '4d. PH',
    '5a. Anwar Ibrahim', '5b. Muhyiddin Yassin', '5c. Zahid Hamidi',
    '5d. Hadi Awang', '5e. Fadillah Yusof', '5f. Rafizi Ramli',
    '5g. Anthony Loke', '5h. Mohamad Hasan', '5i. Mohamad Sabu',
    '6. Are you satisfied with your current federal government?',
    '7. Are you satisfied with the performance of the current Prime Minister?',
    '8. Which government are you satisfied with?',
    '9. Which party did you vote for in GE15?',
    '10. Which of the following state-level issues is most important to you?',
    '11. Are you satisfied with the State Government’s performance in addressing this issue? [based on answer to Q10]',
    '12. Are you satisfied with the level of development and economic opportunities in your area?',
    '13. What is the most negative legacy of the current State Government?',
    '14. What is the most positive legacy of the current State Government?',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set',
    'untrimmed_weight',
    'trimmed_weight'
]]

In [ ]:
# Temp. code
likert_mapping = {'Very negative': '1) Very negative',
                  'Negative': '2) Negative',
                  'Neutral': '3) Neutral',
                  'Positive': '4) Positive',
                  'Very positive': '5) Very positive',
                  'Not recognize': '6) Not recognize'}

weighted_results_prn.replace(likert_mapping, inplace=True)

In [ ]:
# Export to be run thru Crosstab Generator
weighted_results_prn.to_excel('pkr2023-PRN-Terengganu-all-sets-weighted_v{}.xlsx'.format(formatted_date.replace('/', '')), index=False)

## Combine weighted results (KEDAH)

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

# Adding this code chunk to track progress by date.
today = date.today()
formatted_date = "/" + today.strftime("%d-%m-%Y").replace("-0", "-")
print("Today's date:", formatted_date)

Today's date: /03-5-2023


In [ ]:
weighted_prn = [
 'pkr2023-KEDAH-April-set1-300CR-weighted.xlsx',
 'pkr2023-KEDAH-April-set2-212CR-weighted.xlsx',
 'pkr2023-KEDAH-April-set3-114CR-weighted.xlsx',
 'pkr2023-KEDAH-April-set4-141CR-weighted.xlsx',
 'pkr2023-KEDAH-April-set5-108CR-weighted.xlsx',
 'pkr2023-KEDAH-April-set6-156CR-weighted.xlsx',
 'pkr2023-KEDAH-April-set7-186CR-weighted.xlsx',
 'pkr2023-KEDAH-April-set8-107CR-weighted.xlsx'
]

In [ ]:
# Loop thru all weighted files and combine
weighted_list_prn = []

for f in weighted_prn:
  df = pd.read_excel(f)
  weighted_list_prn.append(df)

weighted_results_prn = pd.concat(weighted_list_prn, axis='index')

In [ ]:
# Rearrange columns
weighted_results_prn = weighted_results_prn[[
    'phonenum',
    '1. Which of the following issues is most important to you?',
    '2. Are you satisfied with your current economic situation?',
    '4a. BERSATU', '4b. PAS', '4c. UMNO', '4d. PH',
    '5a. Anwar Ibrahim', '5b. Muhyiddin Yassin', '5c. Zahid Hamidi',
    '5d. Hadi Awang', '5e. Fadillah Yusof', '5f. Rafizi Ramli',
    '5g. Anthony Loke', '5h. Mohamad Hasan', '5i. Mohamad Sabu',
    '6. Are you satisfied with your current federal government?',
    '7. Are you satisfied with the performance of the current Prime Minister?',
    '8. Which government are you satisfied with?',
    '9. Which party did you vote for in GE15?',
    '10. Which of the following state-level issues is most important to you?',
    '11. Are you satisfied with the State Government’s performance in addressing this issue? [based on answer to Q10]',
    '12. Are you satisfied with the level of development and economic opportunities in your area?',
    '13. What is the most negative legacy of the current State Government?',
    '14. What is the most positive legacy of the current State Government?',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set',
    'untrimmed_weight',
    'trimmed_weight'
]]

In [ ]:
# Temp. code
likert_mapping = {'Very negative': '1) Very negative',
                  'Negative': '2) Negative',
                  'Neutral': '3) Neutral',
                  'Positive': '4) Positive',
                  'Very positive': '5) Very positive',
                  'Not recognize': '6) Not recognize'}

weighted_results_prn.replace(likert_mapping, inplace=True)

In [ ]:
# Export to be run thru Crosstab Generator
weighted_results_prn.to_excel('pkr2023-PRN-KEDAH-all-sets-weighted_v{}.xlsx'.format(formatted_date.replace('/', '')), index=False)

## Combine weighted results (NEGERI SEMBILAN)

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

# Adding this code chunk to track progress by date.
today = date.today()
formatted_date = "/" + today.strftime("%d-%m-%Y").replace("-0", "-")
print("Today's date:", formatted_date)

Today's date: /03-5-2023


In [ ]:
weighted_prn = [
 'pkr2023-Negeri-Sembilan-April-set1-44CR-weighted.xlsx',
 'pkr2023-Negeri-Sembilan-April-set2-65CR-weighted.xlsx',
 'pkr2023-Negeri-Sembilan-April-set3-23CR-weighted.xlsx',
 'pkr2023-Negeri-Sembilan-April-set4-49CR-weighted.xlsx',
 'pkr2023-Negeri-Sembilan-April-set5-35CR-weighted.xlsx',
 'pkr2023-Negeri-Sembilan-April-set6-44CR-weighted.xlsx',
 'pkr2023-Negeri-Sembilan-April-set7-36CR-weighted.xlsx',
 'pkr2023-Negeri-Sembilan-April-set8-46CR-weighted.xlsx'
]

In [ ]:
# Loop thru all weighted files and combine
weighted_list_prn = []

for f in weighted_prn:
  df = pd.read_excel(f)
  weighted_list_prn.append(df)

weighted_results_prn = pd.concat(weighted_list_prn, axis='index')

In [ ]:
# Rearrange columns
weighted_results_prn = weighted_results_prn[[
    'phonenum',
    '1. Which of the following issues is most important to you?',
    '2. Are you satisfied with your current economic situation?',
    '4a. BERSATU', '4b. PAS', '4c. UMNO', '4d. PH',
    '5a. Anwar Ibrahim', '5b. Muhyiddin Yassin', '5c. Zahid Hamidi',
    '5d. Hadi Awang', '5e. Fadillah Yusof', '5f. Rafizi Ramli',
    '5g. Anthony Loke', '5h. Mohamad Hasan', '5i. Mohamad Sabu',
    '6. Are you satisfied with your current federal government?',
    '7. Are you satisfied with the performance of the current Prime Minister?',
    '8. Which government are you satisfied with?',
    '9. Which party did you vote for in GE15?',
    '10. Which of the following state-level issues is most important to you?',
    '11. Are you satisfied with the State Government’s performance in addressing this issue? [based on answer to Q10]',
    '12. Are you satisfied with the level of development and economic opportunities in your area?',
    '13. What is the most negative legacy of the current State Government?',
    '14. What is the most positive legacy of the current State Government?',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set',
    'untrimmed_weight',
    'trimmed_weight'
]]

In [ ]:
# Temp. code
likert_mapping = {'Very negative': '1) Very negative',
                  'Negative': '2) Negative',
                  'Neutral': '3) Neutral',
                  'Positive': '4) Positive',
                  'Very positive': '5) Very positive',
                  'Not recognize': '6) Not recognize'}

weighted_results_prn.replace(likert_mapping, inplace=True)

In [ ]:
# Export to be run thru Crosstab Generator
weighted_results_prn.to_excel('pkr2023-PRN-Negeri-9-all-sets-weighted_v{}.xlsx'.format(formatted_date.replace('/', '')), index=False)

## Combine weighted results (Pulau Pinang)

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

# Adding this code chunk to track progress by date.
today = date.today()
formatted_date = "/" + today.strftime("%d-%m-%Y").replace("-0", "-")
print("Today's date:", formatted_date)

Today's date: /03-5-2023


In [ ]:
weighted_prn = [
 'pkr2023-Pulau-Pinang-April-set1-144CR-weighted.xlsx',
 'pkr2023-Pulau-Pinang-April-set2-137CR-weighted.xlsx',
 'pkr2023-Pulau-Pinang-April-set3-54CR-weighted.xlsx',
 'pkr2023-Pulau-Pinang-April-set4-91CR-weighted.xlsx',
 'pkr2023-Pulau-Pinang-April-set5-66CR-weighted.xlsx',
 'pkr2023-Pulau-Pinang-April-set6-90CR-weighted.xlsx',
 'pkr2023-Pulau-Pinang-April-set7-80CR-weighted.xlsx',
 'pkr2023-Pulau-Pinang-April-set8-70CR-weighted.xlsx'
]

In [ ]:
# Loop thru all weighted files and combine
weighted_list_prn = []

for f in weighted_prn:
  df = pd.read_excel(f)
  weighted_list_prn.append(df)

weighted_results_prn = pd.concat(weighted_list_prn, axis='index')

In [ ]:
# Rearrange columns
weighted_results_prn = weighted_results_prn[[
    'phonenum',
    '1. Which of the following issues is most important to you?',
    '2. Are you satisfied with your current economic situation?',
    '4a. BERSATU', '4b. PAS', '4c. UMNO', '4d. PH',
    '5a. Anwar Ibrahim', '5b. Muhyiddin Yassin', '5c. Zahid Hamidi',
    '5d. Hadi Awang', '5e. Fadillah Yusof', '5f. Rafizi Ramli',
    '5g. Anthony Loke', '5h. Mohamad Hasan', '5i. Mohamad Sabu',
    '6. Are you satisfied with your current federal government?',
    '7. Are you satisfied with the performance of the current Prime Minister?',
    '8. Which government are you satisfied with?',
    '9. Which party did you vote for in GE15?',
    '10. Which of the following state-level issues is most important to you?',
    '11. Are you satisfied with the State Government’s performance in addressing this issue? [based on answer to Q10]',
    '12. Are you satisfied with the level of development and economic opportunities in your area?',
    '13. What is the most negative legacy of the current State Government?',
    '14. What is the most positive legacy of the current State Government?',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set',
    'untrimmed_weight',
    'trimmed_weight'
]]

In [ ]:
# Temp. code
likert_mapping = {'Very negative': '1) Very negative',
                  'Negative': '2) Negative',
                  'Neutral': '3) Neutral',
                  'Positive': '4) Positive',
                  'Very positive': '5) Very positive',
                  'Not recognize': '6) Not recognize'}

weighted_results_prn.replace(likert_mapping, inplace=True)

In [ ]:
# Export to be run thru Crosstab Generator
weighted_results_prn.to_excel('pkr2023-PRN-Pulau-Pinang-all-sets-weighted_v{}.xlsx'.format(formatted_date.replace('/', '')), index=False)

## Combine weighted results (Selangor)

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

# Adding this code chunk to track progress by date.
today = date.today()
formatted_date = "/" + today.strftime("%d-%m-%Y").replace("-0", "-")
print("Today's date:", formatted_date)

Today's date: /03-5-2023


In [ ]:
weighted_prn = [
 'pkr2023-Selangor-April-set1-142CR-weighted.xlsx',
 'pkr2023-Selangor-April-set2-236CR-weighted.xlsx',
 'pkr2023-Selangor-April-set3-129CR-weighted.xlsx',
 'pkr2023-Selangor-April-set4-164CR-weighted.xlsx',
 'pkr2023-Selangor-April-set5-146CR-weighted.xlsx',
 'pkr2023-Selangor-April-set6-171CR-weighted.xlsx',
 'pkr2023-Selangor-April-set7-173CR-weighted.xlsx',
 'pkr2023-Selangor-April-set8-156CR-weighted.xlsx'
]

In [ ]:
# Loop thru all weighted files and combine
weighted_list_prn = []

for f in weighted_prn:
  df = pd.read_excel(f)
  weighted_list_prn.append(df)

weighted_results_prn = pd.concat(weighted_list_prn, axis='index')

In [ ]:
# Rearrange columns
weighted_results_prn = weighted_results_prn[[
    'phonenum',
    '1. Which of the following issues is most important to you?',
    '2. Are you satisfied with your current economic situation?',
    '4a. BERSATU', '4b. PAS', '4c. UMNO', '4d. PH',
    '5a. Anwar Ibrahim', '5b. Muhyiddin Yassin', '5c. Zahid Hamidi',
    '5d. Hadi Awang', '5e. Fadillah Yusof', '5f. Rafizi Ramli',
    '5g. Anthony Loke', '5h. Mohamad Hasan', '5i. Mohamad Sabu',
    '6. Are you satisfied with your current federal government?',
    '7. Are you satisfied with the performance of the current Prime Minister?',
    '8. Which government are you satisfied with?',
    '9. Which party did you vote for in GE15?',
    '10. Which of the following state-level issues is most important to you?',
    '11. Are you satisfied with the State Government’s performance in addressing this issue? [based on answer to Q10]',
    '12. Are you satisfied with the level of development and economic opportunities in your area?',
    '13. What is the most negative legacy of the current State Government?',
    '14. What is the most positive legacy of the current State Government?',
    'ethgroup',
    'agegroup',
    'gender',
    'urbanity',
    'State',
    'Region',
    'Set',
    'untrimmed_weight',
    'trimmed_weight'
]]

In [ ]:
# Temp. code
likert_mapping = {'Very negative': '1) Very negative',
                  'Negative': '2) Negative',
                  'Neutral': '3) Neutral',
                  'Positive': '4) Positive',
                  'Very positive': '5) Very positive',
                  'Not recognize': '6) Not recognize'}

weighted_results_prn.replace(likert_mapping, inplace=True)

In [ ]:
# Export to be run thru Crosstab Generator
weighted_results_prn.to_excel('pkr2023-PRN-Selangor-all-sets-weighted_v{}.xlsx'.format(formatted_date.replace('/', '')), index=False)